In [2]:
import numpy as np
import pandas as pd
import requests
import csv
import ssl
from urllib.request import urlopen
from urllib.parse import quote_plus
from konlpy.tag import Okt
from collections import Counter

In [3]:
okt = Okt()

In [3]:
pd.set_option('display.max_columns', None)

# Skirt 리뷰 전처리

In [5]:
df_skirt = pd.read_csv('./musinsa_review_skirt.csv')

In [6]:
df_skirt.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021.10.14,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021.08.22,2029080,0,1,159cm,60kg,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021.08.12,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021.08.06,1911322,0,1,158cm,54kg,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021.07.07,1911322,0,1,159cm,44kg,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0


In [339]:
df_skirt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12391 entries, 0 to 12390
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   userName       12375 non-null  object
 1   date           12391 non-null  object
 2   goodsNo        12391 non-null  int64 
 3   userSexMen     12391 non-null  int64 
 4   userSexWomen   12391 non-null  int64 
 5   userHeight     12391 non-null  object
 6   userWeight     12391 non-null  object
 7   goodsSize      12391 non-null  object
 8   reviewContent  12391 non-null  object
 9   reviewImg      3049 non-null   object
 10  reviewStyle    12391 non-null  int64 
 11  size           12391 non-null  int64 
 12  bright         12391 non-null  int64 
 13  color          12391 non-null  int64 
 14  thickness      12391 non-null  int64 
 15  weightness     12391 non-null  int64 
 16  touch          12391 non-null  int64 
 17  helpNo         12391 non-null  int64 
 18  styleLikeNo    12391 non-n

In [340]:
df_skirt.shape

(12391, 19)

### 탈퇴 인원 제거

In [341]:
df_skirt.dropna(subset="userName", axis=0, inplace=True)
df_skirt = df_skirt.reset_index(drop=True)

In [342]:
df_skirt.shape

(12375, 19)

### Date를 년, 월, 일로 등분

In [343]:
df_skirt.dropna(subset="date", axis=0, inplace=True)

In [344]:
df_skirt.shape

(12375, 19)

In [345]:
df_skirt.insert(2, "dateDay", 0, True)
df_skirt.insert(2, "dateMonth", 0, True)
df_skirt.insert(2, "dateYear", 0, True)

In [346]:
df_skirt.columns

Index(['userName', 'date', 'dateYear', 'dateMonth', 'dateDay', 'goodsNo',
       'userSexMen', 'userSexWomen', 'userHeight', 'userWeight', 'goodsSize',
       'reviewContent', 'reviewImg', 'reviewStyle', 'size', 'bright', 'color',
       'thickness', 'weightness', 'touch', 'helpNo', 'styleLikeNo'],
      dtype='object')

In [347]:
df_skirt['dateYear'] = df_skirt.apply(lambda x: x['date'].split('.')[0], axis=1)
df_skirt['dateMonth'] = df_skirt.apply(lambda x: x['date'].split('.')[1], axis=1)
df_skirt['dateDay'] = df_skirt.apply(lambda x: x['date'].split('.')[2], axis=1)

In [348]:
df_skirt.head()

,userName,date,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021.10.14,2021,10,14,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021.08.22,2021,08,22,2029080,0,1,159cm,60kg,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021.08.12,2021,08,12,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021.08.06,2021,08,06,1911322,0,1,158cm,54kg,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021.07.07,2021,07,07,1911322,0,1,159cm,44kg,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0


In [349]:
df_skirt = df_skirt.drop('date', axis=1)

In [350]:
df_skirt.dateYear = df_skirt.dateYear.astype('int')
df_skirt.dateMonth = df_skirt.dateMonth.astype('int')
df_skirt.dateDay = df_skirt.dateDay.astype('int')

### goodsNo

In [351]:
df_skirt.dropna(subset="goodsNo", axis=0, inplace=True)

In [352]:
df_skirt.shape

(12375, 21)

### 성별

In [353]:
df_skirt[df_skirt.userSexMen == 1].shape

(192, 21)

In [354]:
df_skirt[df_skirt.userSexWomen==1].shape

(12183, 21)

In [355]:
192+12183

12375

### 키와 몸무게

In [356]:
df_skirt.dropna(subset=["userHeight", "userWeight"], axis=0, inplace=True)

In [357]:
df_skirt.shape

(12375, 21)

In [358]:
df_skirt.userHeight = df_skirt.userHeight.apply(lambda x: x.replace("cm", "").replace(" ", ""))
df_skirt.userHeight = df_skirt.userHeight.astype('int64')

In [359]:
df_skirt.userWeight = df_skirt.userWeight.apply(lambda x: x.replace("kg", "").replace(" ", ""))
df_skirt.userWeight = df_skirt.userWeight.astype('int64')

In [360]:
a = list((df_skirt[df_skirt.userHeight==0][df_skirt.userWeight==0]).index)
a

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12588\3390044186.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = list((df_skirt[df_skirt.userHeight==0][df_skirt.userWeight==0]).index)


[]

In [361]:
df_skirt = df_skirt.drop(a)
df_skirt.reset_index(drop=True)

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021,10,14,2029080,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021,8,22,2029080,0,1,159,60,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021,8,12,2029080,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021,8,6,1911322,0,1,158,54,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021,7,7,1911322,0,1,159,44,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12370,뉴비_54bed8fe,2022,3,6,2340070,0,1,160,48,M,"['보통 여기 치마는 m 사면 무난하게 맞았는데', <br/>, '얘는 좀 허리가 ...",NaN,0,1,2,2,0,0,0,0,0
12371,얼래프,2022,3,3,2340070,0,1,163,47,S,['패턴은 특이하고 이쁜데 몸매 딱 잡아주는 슬림핏은 아니고 살짝 어벙벙해요 ㅠ'],NaN,0,1,2,2,0,0,0,0,0
12372,교촌한입만,2022,2,25,2340070,0,1,160,50,S,"['엉덩이 있는 편인데 S 잘맞아요', <br/>, '허리는 25-26 정도인데 적...",NaN,0,2,1,1,0,0,0,0,0
12373,야옹웅,2022,2,18,2340070,0,1,158,50,M,['실물로 보는게 더 이쁜거 같아요 진짜 이쁩니다 대박'],NaN,0,2,2,1,0,0,0,0,0


### 이미지

In [362]:
df_skirt.reviewStyle.value_counts()

0    9328
1    3047
Name: reviewStyle, dtype: int64

In [363]:
df_skirt.reviewImg = df_skirt.reviewImg.fillna("-")

In [364]:
df_skirt.reviewImg

0        https://image.msscdn.net/data/estimate/2029080...
1                                                        -
2                                                        -
3        https://image.msscdn.net/data/estimate/1911322...
4        https://image.msscdn.net/data/estimate/1911322...
                               ...                        
12370                                                    -
12371                                                    -
12372                                                    -
12373                                                    -
12374                                                    -
Name: reviewImg, Length: 12375, dtype: object

In [365]:
type(df_skirt.reviewImg[1])

str

### Contents

In [366]:
print(df_skirt.reviewContent[0])
print(df_skirt.reviewContent[1].split(","))

['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용']
["['생각보다 짧은 치마'", ' <br/>', " '제 체격에는 좀 작게 느껴질만한 사이즈'", ' <br/>', " '디자인은 되게 예쁨']"]


In [367]:
cnt = 0
result = []
for i in df_skirt.reviewContent:
    if '<span>' in i:
        cnt += 1
        result.append(i)
print(cnt)

2


In [368]:
df_skirt[df_skirt.reviewContent=='[\'데님 색이 너무 맘에 들어서 구매했습니다!\', <br/>, \'키가 작아서 길이가 너무 길면 어떡하나 싶었는데 길이도 잘 맞고 허리도 너무 잘 맞았습니다>_\', \'!</div\', \'\\n\', <div class="review-evaluation">\n<ul class="review-evaluation__list">\n<li class="review-evaluation__item">사이즈 <span>보통이에요</span></li>\n<li class="review-evaluation__item">밝기 <span>보통이에요</span></li>\n<li class="review-evaluation__item">색감 <span>보통이에요</span></li>\n</ul>\n</div>, \'\\n\', \' 이미지 \', \'\\n\', <div class="review-content-photo">\n<div class="review-content-photo__wrap">\n<ul class="review-content-photo__list">\n<li class="review-content-photo__item" data-img-index="0" style="background-image:url(\'//image.msscdn.net/data/estimate/1618698_0/gallery_607d08ce95b27.jpg.list\');">\n<img alt="밀리언코르(MILLIONCOR) [DANA 8130]커버밴드 A라인 데님 스커트(블랙그레이) 후기" src="//image.msscdn.net/data/estimate/1618698_0/gallery_607d08ce95b27.jpg.view"/>\n</li>\n</ul>\n</div>\n</div>, \'\\n\', \' 이모티콘 \', \'\\n\', \' SNS 공유시 필요한 데이터 \', \'\\n\', <span class="p_name" style="display: none;">[DANA 8130]커버밴드 A라인 데님 스커트(블랙그레이)</span>, \'\\n\']']

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
1184,여신예리,2021,4,19,1618698,0,1,152,53,M,"['데님 색이 너무 맘에 들어서 구매했습니다!', <br/>, '키가 작아서 길이가...",https://image.msscdn.net/data/estimate/1618698...,1,2,2,2,0,0,0,0,0


In [369]:
df_skirt[df_skirt.reviewContent=='[\'허리 밴딩이 넓어서 좋고 싸이즈도 은근 생각보다 커서 S 사이즈 사게 잘한거 같애요!! 예뻐요~>\', \'!</div\', \'\\n\', <div class="review-evaluation">\n<ul class="review-evaluation__list">\n<li class="review-evaluation__item">사이즈 <span>보통이에요</span></li>\n<li class="review-evaluation__item">밝기 <span>보통이에요</span></li>\n<li class="review-evaluation__item">색감 <span>보통이에요</span></li>\n</ul>\n</div>, \'\\n\', \' 이미지 \', \'\\n\', \' 이모티콘 \', \'\\n\', \' SNS 공유시 필요한 데이터 \', \'\\n\', <span class="p_name" style="display: none;">Inside-Out Kangaroo Skirt Melange-Reverse</span>, \'\\n\']']

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
11323,써드3,2021,7,23,1788798,0,1,163,50,S,['허리 밴딩이 넓어서 좋고 싸이즈도 은근 생각보다 커서 S 사이즈 사게 잘한거 같...,-,0,2,2,2,0,0,0,0,0


In [370]:
df_skirt.loc[1186, 'reviewContent'] = result[0].split('</div')[0][:]
df_skirt.loc[11338, 'reviewContent'] = result[1].split('</div')[0][:]

In [371]:
print(df_skirt.loc[1186].reviewContent)
print(df_skirt.loc[11338].reviewContent)

['데님 색이 너무 맘에 들어서 구매했습니다!', <br/>, '키가 작아서 길이가 너무 길면 어떡하나 싶었는데 길이도 잘 맞고 허리도 너무 잘 맞았습니다>_', '!
['허리 밴딩이 넓어서 좋고 싸이즈도 은근 생각보다 커서 S 사이즈 사게 잘한거 같애요!! 예뻐요~>', '!


In [372]:
df_skirt.reviewContent = df_skirt.reviewContent.apply(lambda x: x.replace("[", "").replace("]", "").replace(" '", "").replace("'", "").replace(", <br/>", "").split(","))

In [373]:
print(df_skirt.reviewContent[0])
print(df_skirt.reviewContent[1])

['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용']
['생각보다 짧은 치마', '제 체격에는 좀 작게 느껴질만한 사이즈', '디자인은 되게 예쁨']


In [374]:
type(df_skirt.reviewContent.loc[0])

list

### 사이즈

In [375]:
sub = df_skirt[["goodsNo", "goodsSize"]]
sub = sub.drop_duplicates(subset = ["goodsNo", "goodsSize"]).reset_index(drop=True)
sub.dropna(subset="goodsNo", inplace=True)
sub.insert(2, 'newGoodsNo', sub.goodsNo, True)
sub.insert(3, 'index', sub.index[:], True)
# sub.newGoodsNo = sub.newGoodsNo.apply(lambda x: round(x))

In [376]:
sub.goodsNo = sub.goodsNo.astype('str')
sub['index'] = sub['index'].astype('str')
sub.newGoodsNo = sub.newGoodsNo.astype('str')

In [377]:
for i in range(2654):
    if len(sub.newGoodsNo[i]) < 7:
        while len(sub.newGoodsNo[i]) < 7:
            sub.newGoodsNo[i] += '0'
sub

,goodsNo,goodsSize,newGoodsNo,index
0,2029080,KHAKI/FREE,2029080,0
1,1911322,M,1911322,1
2,1911322,S,1911322,2
3,1625877,FREE,1625877,3
4,1586158,L(28-30),1586158,4
...,...,...,...,...
2649,2117256,S,2117256,2649
2650,2117256,M,2117256,2650
2651,2117256,L,2117256,2651
2652,2340070,M,2340070,2652


In [378]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2654 entries, 0 to 2653
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   goodsNo     2654 non-null   object
 1   goodsSize   2654 non-null   object
 2   newGoodsNo  2654 non-null   object
 3   index       2654 non-null   object
dtypes: object(4)
memory usage: 83.1+ KB


In [379]:
sub['newGoodsNo'] = sub.apply(lambda x: x['newGoodsNo']+x['index'], axis=1)
sub

,goodsNo,goodsSize,newGoodsNo,index
0,2029080,KHAKI/FREE,20290800,0
1,1911322,M,19113221,1
2,1911322,S,19113222,2
3,1625877,FREE,16258773,3
4,1586158,L(28-30),15861584,4
...,...,...,...,...
2649,2117256,S,21172562649,2649
2650,2117256,M,21172562650,2650
2651,2117256,L,21172562651,2651
2652,2340070,M,23400702652,2652


In [380]:
sub.goodsNo = sub.goodsNo.astype('int')
sub.newGoodsNo = sub.newGoodsNo.astype('int64')

In [381]:
df_skirt

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021,10,14,2029080,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021,8,22,2029080,0,1,159,60,KHAKI/FREE,"[생각보다 짧은 치마, 제 체격에는 좀 작게 느껴질만한 사이즈, 디자인은 되게 예쁨]",-,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021,8,12,2029080,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],-,0,2,2,2,0,0,0,0,0
3,미역이233,2021,8,6,1911322,0,1,158,54,M,"[주름이 많이 가긴 하는데 스판도 좋고 이쁩니다, 제 키에는 좀 길어서 무릎까지 오...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021,7,7,1911322,0,1,159,44,S,[리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나고...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12370,뉴비_54bed8fe,2022,3,6,2340070,0,1,160,48,M,"[보통 여기 치마는 m 사면 무난하게 맞았는데, 얘는 좀 허리가 크네요, 그래도 상...",-,0,1,2,2,0,0,0,0,0
12371,얼래프,2022,3,3,2340070,0,1,163,47,S,[패턴은 특이하고 이쁜데 몸매 딱 잡아주는 슬림핏은 아니고 살짝 어벙벙해요 ㅠ],-,0,1,2,2,0,0,0,0,0
12372,교촌한입만,2022,2,25,2340070,0,1,160,50,S,"[엉덩이 있는 편인데 S 잘맞아요, 허리는 25-26 정도인데 적당히 맞는편]",-,0,2,1,1,0,0,0,0,0
12373,야옹웅,2022,2,18,2340070,0,1,158,50,M,[실물로 보는게 더 이쁜거 같아요 진짜 이쁩니다 대박],-,0,2,2,1,0,0,0,0,0


#### month 당 사는 옷 수

In [382]:
df_skirt.insert(5, 'month12', 0, True)
df_skirt.insert(5, 'month11', 0, True)
df_skirt.insert(5, 'month10', 0, True)
df_skirt.insert(5, 'month9', 0, True)
df_skirt.insert(5, 'month8', 0, True)
df_skirt.insert(5, 'month7', 0, True)
df_skirt.insert(5, 'month6', 0, True)
df_skirt.insert(5, 'month5', 0, True)
df_skirt.insert(5, 'month4', 0, True)
df_skirt.insert(5, 'month3', 0, True)
df_skirt.insert(5, 'month2', 0, True)
df_skirt.insert(5, 'month1', 0, True)

In [383]:
df_skirt.month1 = df_skirt.apply(lambda x: 1 if x.dateMonth==1 else x.month1, axis=1)
df_skirt.month2 = df_skirt.apply(lambda x: 1 if x.dateMonth==2 else x.month2, axis=1)
df_skirt.month3 = df_skirt.apply(lambda x: 1 if x.dateMonth==3 else x.month3, axis=1)
df_skirt.month4 = df_skirt.apply(lambda x: 1 if x.dateMonth==4 else x.month4, axis=1)
df_skirt.month5 = df_skirt.apply(lambda x: 1 if x.dateMonth==5 else x.month5, axis=1)
df_skirt.month6 = df_skirt.apply(lambda x: 1 if x.dateMonth==6 else x.month6, axis=1)
df_skirt.month7 = df_skirt.apply(lambda x: 1 if x.dateMonth==7 else x.month7, axis=1)
df_skirt.month8 = df_skirt.apply(lambda x: 1 if x.dateMonth==8 else x.month8, axis=1)
df_skirt.month9 = df_skirt.apply(lambda x: 1 if x.dateMonth==9 else x.month9, axis=1)
df_skirt.month10 = df_skirt.apply(lambda x: 1 if x.dateMonth==10 else x.month10, axis=1)
df_skirt.month11 = df_skirt.apply(lambda x: 1 if x.dateMonth==11 else x.month11, axis=1)
df_skirt.month12 = df_skirt.apply(lambda x: 1 if x.dateMonth==12 else x.month12, axis=1)

In [384]:
from copy import deepcopy
a = deepcopy(df_skirt)

In [385]:
k = a.groupby(['goodsNo'])['reviewContent'].apply(list).reset_index(name="reviewContents")
k

,goodsNo,reviewContents
0,387628,[[리본이 포인트네요. 길이도 너무 짧지 않아 마음에 듭니다.]]
1,387631,[[생각보다 진한색이고 허리가 꽉맞아서 살짝벌어지네요ㅠㅠ]]
2,387632,[[디자인이 마음에 듭니다^^ 입기도 편하구요 완전 마음에 듭니다🙂👍]]
3,407626,"[[배송은 빨리왔고, 색감이 정말 예쁘네요~, 옷의 핏감은 솔직히 별로예요~]]"
4,463655,"[[앞엔볼만한데 뒷부분은 허리밴드쭈글쭈글 너무크고 싸니까ㅜㅜ롱티랑], [허리는 77..."
...,...,...
1795,2389731,[[밴딩이라 편해요 길이감도 괜찮구요 전반적으로 만족해요]]
1796,2391576,[[와우 흔하지 않은 스타일이라 너무 만족해요!! 치마 자체도 너무 예쁘구요!! 처...
1797,2391598,[[예쁘고 탄탄해요 핏도 괜찮습니다 나쁘지 않은 것 같아요]]
1798,2394978,[[이뻐요 전체적으로 색상똑같아요 예쁩니다 힣힣]]


In [386]:
k.reviewContents = k.reviewContents.astype('str')
k.reviewContents = k.apply(lambda x: x.reviewContents.replace("[", "").replace("]", ""), axis=1)

In [387]:
k.insert(2, 'reviewNoun1', '', True)
k.reviewNoun1 = k.apply(lambda x: Counter(okt.nouns(x.reviewContents)).most_common(100), axis=1)
k

,goodsNo,reviewContents,reviewNoun1
0,387628,'리본이 포인트네요. 길이도 너무 짧지 않아 마음에 듭니다.',"[(리본, 1), (포인트, 1), (길이, 1), (마음, 1), (듭니, 1)]"
1,387631,'생각보다 진한색이고 허리가 꽉맞아서 살짝벌어지네요ㅠㅠ',"[(생각, 1), (색, 1), (허리, 1), (꽉, 1), (살짝, 1)]"
2,387632,'디자인이 마음에 듭니다^^ 입기도 편하구요 완전 마음에 듭니다🙂👍',"[(마음, 2), (듭니, 2), (디자인, 1), (완전, 1)]"
3,407626,"'배송은 빨리왔고', '색감이 정말 예쁘네요~', '옷의 핏감은 솔직히 별로예요~'","[(배송, 1), (색감, 1), (정말, 1), (옷, 1), (핏감, 1), (..."
4,463655,"'앞엔볼만한데 뒷부분은 허리밴드쭈글쭈글 너무크고 싸니까ㅜㅜ롱티랑', '허리는 77은...","[(허리, 2), (앞, 1), (볼, 1), (부분, 1), (밴드, 1), (쭈..."
...,...,...,...
1795,2389731,'밴딩이라 편해요 길이감도 괜찮구요 전반적으로 만족해요',"[(밴딩, 1), (길이, 1), (전반, 1)]"
1796,2391576,'와우 흔하지 않은 스타일이라 너무 만족해요!! 치마 자체도 너무 예쁘구요!! 처음...,"[(와우, 1), (스타일, 1), (치마, 1), (자체, 1), (처음, 1),..."
1797,2391598,'예쁘고 탄탄해요 핏도 괜찮습니다 나쁘지 않은 것 같아요',"[(핏, 1), (것, 1)]"
1798,2394978,'이뻐요 전체적으로 색상똑같아요 예쁩니다 힣힣',"[(전체, 1), (색상, 1)]"


In [388]:
k.insert(3, 'reviewNoun', '', True)

In [389]:
for i in k.itertuples():
    kkk = []
    for x, y in k.reviewNoun1[i.Index]:
        kkkkk = {'text': x, 'value': y}
        kkk.append(kkkkk)
    k.at[i.Index, 'reviewNoun'] = kkk
k

,goodsNo,reviewContents,reviewNoun1,reviewNoun
0,387628,'리본이 포인트네요. 길이도 너무 짧지 않아 마음에 듭니다.',"[(리본, 1), (포인트, 1), (길이, 1), (마음, 1), (듭니, 1)]","[{'text': '리본', 'value': 1}, {'text': '포인트', '..."
1,387631,'생각보다 진한색이고 허리가 꽉맞아서 살짝벌어지네요ㅠㅠ',"[(생각, 1), (색, 1), (허리, 1), (꽉, 1), (살짝, 1)]","[{'text': '생각', 'value': 1}, {'text': '색', 'va..."
2,387632,'디자인이 마음에 듭니다^^ 입기도 편하구요 완전 마음에 듭니다🙂👍',"[(마음, 2), (듭니, 2), (디자인, 1), (완전, 1)]","[{'text': '마음', 'value': 2}, {'text': '듭니', 'v..."
3,407626,"'배송은 빨리왔고', '색감이 정말 예쁘네요~', '옷의 핏감은 솔직히 별로예요~'","[(배송, 1), (색감, 1), (정말, 1), (옷, 1), (핏감, 1), (...","[{'text': '배송', 'value': 1}, {'text': '색감', 'v..."
4,463655,"'앞엔볼만한데 뒷부분은 허리밴드쭈글쭈글 너무크고 싸니까ㅜㅜ롱티랑', '허리는 77은...","[(허리, 2), (앞, 1), (볼, 1), (부분, 1), (밴드, 1), (쭈...","[{'text': '허리', 'value': 2}, {'text': '앞', 'va..."
...,...,...,...,...
1795,2389731,'밴딩이라 편해요 길이감도 괜찮구요 전반적으로 만족해요',"[(밴딩, 1), (길이, 1), (전반, 1)]","[{'text': '밴딩', 'value': 1}, {'text': '길이', 'v..."
1796,2391576,'와우 흔하지 않은 스타일이라 너무 만족해요!! 치마 자체도 너무 예쁘구요!! 처음...,"[(와우, 1), (스타일, 1), (치마, 1), (자체, 1), (처음, 1),...","[{'text': '와우', 'value': 1}, {'text': '스타일', '..."
1797,2391598,'예쁘고 탄탄해요 핏도 괜찮습니다 나쁘지 않은 것 같아요',"[(핏, 1), (것, 1)]","[{'text': '핏', 'value': 1}, {'text': '것', 'val..."
1798,2394978,'이뻐요 전체적으로 색상똑같아요 예쁩니다 힣힣',"[(전체, 1), (색상, 1)]","[{'text': '전체', 'value': 1}, {'text': '색상', 'v..."


In [390]:
k = k.drop(['reviewContents', 'reviewNoun1'], axis=1)
k

,goodsNo,reviewNoun
0,387628,"[{'text': '리본', 'value': 1}, {'text': '포인트', '..."
1,387631,"[{'text': '생각', 'value': 1}, {'text': '색', 'va..."
2,387632,"[{'text': '마음', 'value': 2}, {'text': '듭니', 'v..."
3,407626,"[{'text': '배송', 'value': 1}, {'text': '색감', 'v..."
4,463655,"[{'text': '허리', 'value': 2}, {'text': '앞', 'va..."
...,...,...
1795,2389731,"[{'text': '밴딩', 'value': 1}, {'text': '길이', 'v..."
1796,2391576,"[{'text': '와우', 'value': 1}, {'text': '스타일', '..."
1797,2391598,"[{'text': '핏', 'value': 1}, {'text': '것', 'val..."
1798,2394978,"[{'text': '전체', 'value': 1}, {'text': '색상', 'v..."


In [391]:
sub = pd.merge(left=sub, right=k, how="left", on='goodsNo')
sub

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun
0,2029080,KHAKI/FREE,20290800,0,"[{'text': '좀', 'value': 3}, {'text': '핏', 'val..."
1,1911322,M,19113221,1,"[{'text': '좀', 'value': 5}, {'text': '허리', 'va..."
2,1911322,S,19113222,2,"[{'text': '좀', 'value': 5}, {'text': '허리', 'va..."
3,1625877,FREE,16258773,3,"[{'text': '촉감', 'value': 1}, {'text': '진짜', 'v..."
4,1586158,L(28-30),15861584,4,"[{'text': '거', 'value': 3}, {'text': '정도', 'va..."
...,...,...,...,...,...
2649,2117256,S,21172562649,2649,"[{'text': '허리', 'value': 17}, {'text': '것', 'v..."
2650,2117256,M,21172562650,2650,"[{'text': '허리', 'value': 17}, {'text': '것', 'v..."
2651,2117256,L,21172562651,2651,"[{'text': '허리', 'value': 17}, {'text': '것', 'v..."
2652,2340070,M,23400702652,2652,"[{'text': '허리', 'value': 3}, {'text': '살짝', 'v..."


In [392]:
d = df_skirt.groupby(['goodsNo', 'goodsSize']).sum()[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]
d

,,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
goodsNo,goodsSize,,,,,,,,,,,,
387628,FREE/PINK,0,0,0,0,0,0,0,0,1,0,0,0
387631,FREE/BLUE,0,0,0,0,0,0,0,1,0,0,0,0
387632,FREE/NAVY,0,0,1,0,0,0,0,0,0,0,0,0
407626,FREE,0,0,1,0,0,0,0,0,0,0,0,0
463655,FREE,0,0,2,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2389731,FREE,0,0,1,0,0,0,0,0,0,0,0,0
2391576,S,0,0,1,0,0,0,0,0,0,0,0,0
2391598,S,0,0,1,0,0,0,0,0,0,0,0,0


#### 키, 몸무게 평균 구하기

In [393]:
c = df_skirt.groupby(['goodsNo', 'goodsSize'], as_index=False).mean()[['goodsNo', 'goodsSize', 'userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]
c

,goodsNo,goodsSize,userHeight,userWeight,userSexMen,userSexWomen
0,387628,FREE/PINK,160.0,47.0,0.0,1.0
1,387631,FREE/BLUE,158.0,46.0,0.0,1.0
2,387632,FREE/NAVY,163.0,50.0,0.0,1.0
3,407626,FREE,161.0,54.0,0.0,1.0
4,463655,FREE,150.0,50.0,0.0,1.0
...,...,...,...,...,...,...
2649,2389731,FREE,165.0,55.0,0.0,1.0
2650,2391576,S,161.0,46.0,0.0,1.0
2651,2391598,S,158.0,46.0,0.0,1.0
2652,2394978,M,160.0,55.0,0.0,1.0


In [394]:
sub.sort_values(['goodsNo', 'goodsSize'], inplace=True)
c.sort_values(['goodsNo', 'goodsSize'], inplace=True)
d.sort_values(['goodsNo', 'goodsSize'], inplace=True)
sub

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun
1116,387628,FREE/PINK,38762801116,1116,"[{'text': '리본', 'value': 1}, {'text': '포인트', '..."
1604,387631,FREE/BLUE,38763101604,1604,"[{'text': '생각', 'value': 1}, {'text': '색', 'va..."
1044,387632,FREE/NAVY,38763201044,1044,"[{'text': '마음', 'value': 2}, {'text': '듭니', 'v..."
740,407626,FREE,4076260740,740,"[{'text': '배송', 'value': 1}, {'text': '색감', 'v..."
465,463655,FREE,4636550465,465,"[{'text': '허리', 'value': 2}, {'text': '앞', 'va..."
...,...,...,...,...,...
2219,2389731,FREE,23897312219,2219,"[{'text': '밴딩', 'value': 1}, {'text': '길이', 'v..."
2318,2391576,S,23915762318,2318,"[{'text': '와우', 'value': 1}, {'text': '스타일', '..."
1607,2391598,S,23915981607,1607,"[{'text': '핏', 'value': 1}, {'text': '것', 'val..."
371,2394978,M,2394978371,371,"[{'text': '전체', 'value': 1}, {'text': '색상', 'v..."


In [395]:
d.index = range(len(d))
d

,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,2,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2649,0,0,1,0,0,0,0,0,0,0,0,0
2650,0,0,1,0,0,0,0,0,0,0,0,0
2651,0,0,1,0,0,0,0,0,0,0,0,0
2652,0,0,1,0,0,0,0,0,0,0,0,0


In [396]:
sub = pd.concat([sub, c[['userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]], axis=1)
sub = pd.concat([sub, d], axis=1)
sub.index = range(len(sub))
sub

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,387628,FREE/PINK,38762801116,1116,"[{'text': '리본', 'value': 1}, {'text': '포인트', '...",162.702128,51.744681,0.042553,0.957447,9,10,0,1,0,0,0,0,1,3,11,12
1,387631,FREE/BLUE,38763101604,1604,"[{'text': '생각', 'value': 1}, {'text': '색', 'va...",165.000000,50.000000,0.000000,1.000000,0,0,2,0,0,0,0,0,0,0,0,0
2,387632,FREE/NAVY,38763201044,1044,"[{'text': '마음', 'value': 2}, {'text': '듭니', 'v...",163.000000,50.000000,0.000000,1.000000,2,0,0,0,0,0,0,0,0,0,0,0
3,407626,FREE,4076260740,740,"[{'text': '배송', 'value': 1}, {'text': '색감', 'v...",165.000000,52.000000,0.000000,1.000000,0,1,0,0,0,0,0,0,0,0,0,0
4,463655,FREE,4636550465,465,"[{'text': '허리', 'value': 2}, {'text': '앞', 'va...",161.733333,53.933333,0.066667,0.933333,0,2,0,0,1,3,2,1,1,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649,2389731,FREE,23897312219,2219,"[{'text': '밴딩', 'value': 1}, {'text': '길이', 'v...",155.000000,50.000000,0.000000,1.000000,0,0,0,0,0,0,0,0,0,1,0,0
2650,2391576,S,23915762318,2318,"[{'text': '와우', 'value': 1}, {'text': '스타일', '...",159.625000,47.750000,0.125000,0.875000,4,3,1,0,0,0,0,0,0,0,0,0
2651,2391598,S,23915981607,1607,"[{'text': '핏', 'value': 1}, {'text': '것', 'val...",167.294118,65.647059,0.000000,1.000000,0,0,0,0,1,4,0,0,4,6,1,1
2652,2394978,M,2394978371,371,"[{'text': '전체', 'value': 1}, {'text': '색상', 'v...",164.000000,52.250000,0.000000,1.000000,0,0,0,0,0,0,0,0,0,0,3,1


In [397]:
df_skirt = pd.merge(left=df_skirt, right=sub[['goodsNo', 'goodsSize','newGoodsNo']], how="left", on=['goodsNo', 'goodsSize'])
df_skirt.head()

,userName,dateYear,dateMonth,dateDay,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,k_hnnj,2021,10,14,2029080,0,0,0,0,0,0,0,0,0,1,0,0,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0,20290800
1,ꉂꉂᵔᗜᵔ,2021,8,22,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,159,60,KHAKI/FREE,"[생각보다 짧은 치마, 제 체격에는 좀 작게 느껴질만한 사이즈, 디자인은 되게 예쁨]",-,0,3,2,2,0,0,0,0,0,20290800
2,k_hnnj,2021,8,12,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],-,0,2,2,2,0,0,0,0,0,20290800
3,미역이233,2021,8,6,1911322,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,54,M,"[주름이 많이 가긴 하는데 스판도 좋고 이쁩니다, 제 키에는 좀 길어서 무릎까지 오...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0,19113221
4,블리츠크랭크!,2021,7,7,1911322,0,0,0,0,0,0,1,0,0,0,0,0,0,1,159,44,S,[리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나고...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0,19113222


In [398]:
len(sub['goodsNo'].unique())

1800

In [399]:
sub.shape

(2654, 21)

In [400]:
sub.to_csv('size_skirt.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

In [401]:
sub.to_json('size_skirt.json', orient='records', force_ascii=False)

### 그 외

In [584]:
df_skirt['size'].value_counts()

2    9747
1    1872
3     756
Name: size, dtype: int64

In [585]:
df_skirt.bright.value_counts()

2    10196
3     1211
1      968
Name: bright, dtype: int64

In [586]:
df_skirt.color.value_counts()

2    8745
1    3468
3     162
Name: color, dtype: int64

In [587]:
df_skirt.thickness.value_counts()

0    12372
2        2
1        1
Name: thickness, dtype: int64

In [588]:
df_skirt.weightness.value_counts()

0    12375
Name: weightness, dtype: int64

In [589]:
df_skirt.touch.value_counts()

0    12375
Name: touch, dtype: int64

### helpNo, styleLikeNo는 모두 0이다. 제거 가능성 높아짐

In [590]:
df_skirt.head()

,userName,dateYear,dateMonth,dateDay,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,k_hnnj,2021,10,14,2029080,0,0,0,0,0,0,0,0,0,1,0,0,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0,20290800
1,ꉂꉂᵔᗜᵔ,2021,8,22,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,159,60,KHAKI/FREE,"[생각보다 짧은 치마, 제 체격에는 좀 작게 느껴질만한 사이즈, 디자인은 되게 예쁨]",-,0,3,2,2,0,0,0,0,0,20290800
2,k_hnnj,2021,8,12,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],-,0,2,2,2,0,0,0,0,0,20290800
3,미역이233,2021,8,6,1911322,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,54,M,"[주름이 많이 가긴 하는데 스판도 좋고 이쁩니다, 제 키에는 좀 길어서 무릎까지 오...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0,19113221
4,블리츠크랭크!,2021,7,7,1911322,0,0,0,0,0,0,1,0,0,0,0,0,0,1,159,44,S,[리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나고...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0,19113222


In [980]:
df_skirt.to_csv('review_skirt.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

## User 데이터 생성 (unique한 유저 정보로 수정)

user_skirt = df_skirt[['userName', 'userSexMen', 'userSexWomen']]
user_skirt = user_skirt.drop_duplicates(subset=['userName'])
user_skirt.sort_values('userName', inplace=True)
user_skirt.index = range(len(user_skirt))

mo = df_skirt.groupby('userName').sum()[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]
mo.sort_values('userName', inplace=True)
mo.index = range(len(mo))

hw = df_skirt.groupby('userName').mean()[['userHeight', 'userWeight']]
hw.sort_values('userName', inplace=True)
hw.index = range(len(hw))

user_skirt = pd.concat([user_skirt, hw, mo], axis=1)
user_skirt

user_skirt[user_skirt.userName=="사이먼0112"]

user_skirt[user_skirt.userName=='k_hnnj'].index

## Transaction 데이터 생성(전체 리뷰인 경우로 수정)

transaction_data = df_skirt[['userName', 'newClothNo']]
transaction_data.insert(0, 'userNo', 0, True)

transaction_data.userNo = transaction_data.apply(lambda x: user_skirt[user_skirt.userName==x.userName].index[0], axis=1)

transaction_data = transaction_data.drop('userName', axis=1)
transaction_data

# Onepiece 리뷰 전처리

In [402]:
df_onepiece = pd.read_csv('./musinsa_review_onepiece.csv')

In [403]:
df_onepiece.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,odddr,2021.05.29,1933622,0,1,163cm,53kg,none,['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 ...,https://image.msscdn.net/data/estimate/1933622...,1,1,2,2,0,0,0,0,0
1,odddr,2021.05.29,1933622,0,1,163cm,53kg,none,"['여름에 편하게 입기 좋을것 같아요!', <br/>, '생각보다 많이 파여서 조심...",NaN,0,1,2,2,0,0,0,0,0
2,뉴비_a7b53dc7,2022.02.25,2297184,0,1,160cm,55kg,M,"['길이감 좋고 스타일 꾸안꾸느낌', <br/>, '빨리 날 좋아지면 입고싶음', ...",https://image.msscdn.net/data/estimate/2297184...,1,2,2,2,0,0,0,0,0
3,뉴비_a7b53dc7,2022.02.25,2297184,0,1,160cm,55kg,M,"['여유있게 입으려고 M사이즈 구매', <br/>, '나그랑이라 어깨 안넓어보이고 ...",NaN,0,2,2,2,0,0,0,0,0
4,YnSUGA,2022.02.22,2297184,0,1,152cm,58kg,S,['봄 신상이라 그런가 확실히 얇아요. 키가 작은 편이라 기장 수선해서 입어야 할 ...,NaN,0,2,2,2,0,0,0,0,0


In [404]:
df_onepiece.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8461 entries, 0 to 8460
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   userName       8448 non-null   object
 1   date           8461 non-null   object
 2   goodsNo        8461 non-null   int64 
 3   userSexMen     8461 non-null   int64 
 4   userSexWomen   8461 non-null   int64 
 5   userHeight     8461 non-null   object
 6   userWeight     8461 non-null   object
 7   goodsSize      8461 non-null   object
 8   reviewContent  8461 non-null   object
 9   reviewImg      2117 non-null   object
 10  reviewStyle    8461 non-null   int64 
 11  size           8461 non-null   int64 
 12  bright         8461 non-null   int64 
 13  color          8461 non-null   int64 
 14  thickness      8461 non-null   int64 
 15  weightness     8461 non-null   int64 
 16  touch          8461 non-null   int64 
 17  helpNo         8461 non-null   int64 
 18  styleLikeNo    8461 non-null

In [405]:
df_onepiece.shape

(8461, 19)

### 탈퇴 인원 제거

In [406]:
df_onepiece.dropna(subset="userName", axis=0, inplace=True)
df_onepiece = df_onepiece.reset_index(drop=True)

In [407]:
df_onepiece.shape

(8448, 19)

### Date를 년, 월, 일로 등분

In [408]:
df_onepiece.dropna(subset="date", axis=0, inplace=True)

In [409]:
df_onepiece.shape

(8448, 19)

In [410]:
df_onepiece.insert(2, "dateDay", 0, True)
df_onepiece.insert(2, "dateMonth", 0, True)
df_onepiece.insert(2, "dateYear", 0, True)

In [411]:
df_onepiece.columns

Index(['userName', 'date', 'dateYear', 'dateMonth', 'dateDay', 'goodsNo',
       'userSexMen', 'userSexWomen', 'userHeight', 'userWeight', 'goodsSize',
       'reviewContent', 'reviewImg', 'reviewStyle', 'size', 'bright', 'color',
       'thickness', 'weightness', 'touch', 'helpNo', 'styleLikeNo'],
      dtype='object')

In [412]:
df_onepiece['dateYear'] = df_onepiece.apply(lambda x: x['date'].split('.')[0], axis=1)
df_onepiece['dateMonth'] = df_onepiece.apply(lambda x: x['date'].split('.')[1], axis=1)
df_onepiece['dateDay'] = df_onepiece.apply(lambda x: x['date'].split('.')[2], axis=1)

In [413]:
df_onepiece.head()

,userName,date,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,odddr,2021.05.29,2021,05,29,1933622,0,1,163cm,53kg,none,['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 ...,https://image.msscdn.net/data/estimate/1933622...,1,1,2,2,0,0,0,0,0
1,odddr,2021.05.29,2021,05,29,1933622,0,1,163cm,53kg,none,"['여름에 편하게 입기 좋을것 같아요!', <br/>, '생각보다 많이 파여서 조심...",NaN,0,1,2,2,0,0,0,0,0
2,뉴비_a7b53dc7,2022.02.25,2022,02,25,2297184,0,1,160cm,55kg,M,"['길이감 좋고 스타일 꾸안꾸느낌', <br/>, '빨리 날 좋아지면 입고싶음', ...",https://image.msscdn.net/data/estimate/2297184...,1,2,2,2,0,0,0,0,0
3,뉴비_a7b53dc7,2022.02.25,2022,02,25,2297184,0,1,160cm,55kg,M,"['여유있게 입으려고 M사이즈 구매', <br/>, '나그랑이라 어깨 안넓어보이고 ...",NaN,0,2,2,2,0,0,0,0,0
4,YnSUGA,2022.02.22,2022,02,22,2297184,0,1,152cm,58kg,S,['봄 신상이라 그런가 확실히 얇아요. 키가 작은 편이라 기장 수선해서 입어야 할 ...,NaN,0,2,2,2,0,0,0,0,0


In [414]:
df_onepiece = df_onepiece.drop('date', axis=1)

In [415]:
df_onepiece.dateYear = df_onepiece.dateYear.astype('int')
df_onepiece.dateMonth = df_onepiece.dateMonth.astype('int')
df_onepiece.dateDay = df_onepiece.dateDay.astype('int')

### goodsNo

In [416]:
df_onepiece.dropna(subset="goodsNo", axis=0, inplace=True)

In [417]:
df_onepiece.shape

(8448, 21)

### 성별

In [418]:
df_onepiece[df_onepiece.userSexMen == 1].shape

(144, 21)

In [419]:
df_onepiece[df_onepiece.userSexWomen==1].shape

(8304, 21)

In [420]:
144+8304

8448

### 키와 몸무게

In [421]:
df_onepiece.dropna(subset=["userHeight", "userWeight"], axis=0, inplace=True)

In [422]:
df_onepiece.shape

(8448, 21)

In [423]:
df_onepiece.userHeight = df_onepiece.userHeight.apply(lambda x: x.replace("cm", "").replace(" ", ""))
df_onepiece.userHeight = df_onepiece.userHeight.astype('int64')

In [424]:
df_onepiece.userWeight = df_onepiece.userWeight.apply(lambda x: x.replace("kg", "").replace(" ", ""))
df_onepiece.userWeight = df_onepiece.userWeight.astype('int64')

In [425]:
a = list((df_onepiece[df_onepiece.userHeight==0][df_onepiece.userWeight==0]).index)
a

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12588\1381632839.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = list((df_onepiece[df_onepiece.userHeight==0][df_onepiece.userWeight==0]).index)


[5919]

In [426]:
df_onepiece = df_onepiece.drop(a)
df_onepiece.reset_index(drop=True)

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,odddr,2021,5,29,1933622,0,1,163,53,none,['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 ...,https://image.msscdn.net/data/estimate/1933622...,1,1,2,2,0,0,0,0,0
1,odddr,2021,5,29,1933622,0,1,163,53,none,"['여름에 편하게 입기 좋을것 같아요!', <br/>, '생각보다 많이 파여서 조심...",NaN,0,1,2,2,0,0,0,0,0
2,뉴비_a7b53dc7,2022,2,25,2297184,0,1,160,55,M,"['길이감 좋고 스타일 꾸안꾸느낌', <br/>, '빨리 날 좋아지면 입고싶음', ...",https://image.msscdn.net/data/estimate/2297184...,1,2,2,2,0,0,0,0,0
3,뉴비_a7b53dc7,2022,2,25,2297184,0,1,160,55,M,"['여유있게 입으려고 M사이즈 구매', <br/>, '나그랑이라 어깨 안넓어보이고 ...",NaN,0,2,2,2,0,0,0,0,0
4,YnSUGA,2022,2,22,2297184,0,1,152,58,S,['봄 신상이라 그런가 확실히 얇아요. 키가 작은 편이라 기장 수선해서 입어야 할 ...,NaN,0,2,2,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8442,yu.r,2020,9,4,1440679,0,1,163,52,NONE,['패턴이 너무 이뻐서 구매했는데 생각보다 치마길이는 짧아서 너무 아쉬웠어요 ㅠㅠ ...,NaN,0,3,2,2,0,0,0,0,0
8443,뚝서,2021,8,10,1941295,0,1,160,50,PINK/FREE,"['원피스는 통통하신 분들도 잘 들어갈 정도로 밴딩이 잘 되어 있어요', <br/>...",NaN,0,2,2,1,0,0,0,0,0
8444,뉴비_ed629198,2021,9,4,1464994,0,1,160,48,FREE,['정말정말 만족하는 원피스 입니다 생각한것 보다 훨씬 예쁘고 봄여름가을 상관없이 ...,https://image.msscdn.net/data/estimate/1464994...,1,2,2,2,0,0,0,0,0
8445,소영파크,2022,2,9,1464994,0,1,170,60,FREE,['여리여리해보이고 샤랄라하고 사이즈 프리지만 길이감도 적당'],NaN,0,2,2,2,0,0,0,0,0


### 이미지

In [427]:
df_onepiece.reviewStyle.value_counts()

0    6331
1    2116
Name: reviewStyle, dtype: int64

In [428]:
df_onepiece.reviewImg = df_onepiece.reviewImg.fillna("-")

In [429]:
df_onepiece.reviewImg

0       https://image.msscdn.net/data/estimate/1933622...
1                                                       -
2       https://image.msscdn.net/data/estimate/2297184...
3                                                       -
4                                                       -
                              ...                        
8443                                                    -
8444                                                    -
8445    https://image.msscdn.net/data/estimate/1464994...
8446                                                    -
8447                                                    -
Name: reviewImg, Length: 8447, dtype: object

### Contents

In [430]:
print(df_onepiece.reviewContent[0])
print(df_onepiece.reviewContent[1].split(","))

['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 사이즈 구애받지 않고 다 입으실 수 있을거 같아요! 허리끈이 따로 있어서 다양하게 스타일링 할 수 있어 좋네요!']
["['여름에 편하게 입기 좋을것 같아요!'", ' <br/>', " '생각보다 많이 파여서 조심해야ㅎㅐ요ㅠㅠ'", ' <br/>', " '앞뒤가 언발란스한 기장때문에 심심하지 않아 좋네요!']"]


In [431]:
# span 태그가 들어가있는 리뷰내용 전처리

cnt = 0
result = []
for i in df_onepiece.reviewContent:
    if '<span>' in i:
        cnt += 1
        result.append(i)
print(cnt)

0


In [432]:
df_onepiece.reviewContent = df_onepiece.reviewContent.apply(lambda x: x.replace("[", "").replace("]", "").replace(" '", "").replace("'", "").replace(", <br/>", "").split(","))

In [433]:
print(df_onepiece.reviewContent[0])
print(df_onepiece.reviewContent[1])

['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 사이즈 구애받지 않고 다 입으실 수 있을거 같아요! 허리끈이 따로 있어서 다양하게 스타일링 할 수 있어 좋네요!']
['여름에 편하게 입기 좋을것 같아요!', '생각보다 많이 파여서 조심해야ㅎㅐ요ㅠㅠ', '앞뒤가 언발란스한 기장때문에 심심하지 않아 좋네요!']


### 사이즈

In [434]:
sub = df_onepiece[["goodsNo", "goodsSize"]]
sub = sub.drop_duplicates(subset = ["goodsNo", "goodsSize"]).reset_index(drop=True)
sub.dropna(subset="goodsNo", inplace=True)
sub.insert(2, 'newGoodsNo', sub.goodsNo, True)
sub.insert(3, 'index', 0, True)
# sub.newGoodsNo = sub.newGoodsNo.apply(lambda x: round(x))
sub['index'] = range(2654, 2654+len(sub))
sub.index = range(2654, 2654+len(sub))
sub

,goodsNo,goodsSize,newGoodsNo,index
2654,1933622,none,1933622,2654
2655,2297184,M,2297184,2655
2656,2297184,S,2297184,2656
2657,2297184,XS,2297184,2657
2658,1771393,MINT/FREE,1771393,2658
...,...,...,...,...
4696,1782233,M,1782233,4696
4697,1790851,M,1790851,4697
4698,1440679,NONE,1440679,4698
4699,1941295,PINK/FREE,1941295,4699


In [435]:
sub.goodsNo = sub.goodsNo.astype('str')
sub['index'] = sub['index'].astype('str')
sub.newGoodsNo = sub.newGoodsNo.astype('str')

In [436]:
len(sub)

2047

In [437]:
for i in range(2654, len(sub)+2654):
    if len(sub.newGoodsNo[i]) < 7:
        while len(sub.newGoodsNo[i]) < 7:
            sub.newGoodsNo[i] += '0'
sub

,goodsNo,goodsSize,newGoodsNo,index
2654,1933622,none,1933622,2654
2655,2297184,M,2297184,2655
2656,2297184,S,2297184,2656
2657,2297184,XS,2297184,2657
2658,1771393,MINT/FREE,1771393,2658
...,...,...,...,...
4696,1782233,M,1782233,4696
4697,1790851,M,1790851,4697
4698,1440679,NONE,1440679,4698
4699,1941295,PINK/FREE,1941295,4699


In [438]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2047 entries, 2654 to 4700
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   goodsNo     2047 non-null   object
 1   goodsSize   2047 non-null   object
 2   newGoodsNo  2047 non-null   object
 3   index       2047 non-null   object
dtypes: object(4)
memory usage: 64.1+ KB


In [439]:
sub['newGoodsNo'] = sub.apply(lambda x: x['newGoodsNo']+x['index'], axis=1)
sub

,goodsNo,goodsSize,newGoodsNo,index
2654,1933622,none,19336222654,2654
2655,2297184,M,22971842655,2655
2656,2297184,S,22971842656,2656
2657,2297184,XS,22971842657,2657
2658,1771393,MINT/FREE,17713932658,2658
...,...,...,...,...
4696,1782233,M,17822334696,4696
4697,1790851,M,17908514697,4697
4698,1440679,NONE,14406794698,4698
4699,1941295,PINK/FREE,19412954699,4699


In [440]:
sub.goodsNo = sub.goodsNo.astype('int')
sub.newGoodsNo = sub.newGoodsNo.astype('int64')

#### month 당 사는 옷 수

In [441]:
df_onepiece.insert(5, 'month12', 0, True)
df_onepiece.insert(5, 'month11', 0, True)
df_onepiece.insert(5, 'month10', 0, True)
df_onepiece.insert(5, 'month9', 0, True)
df_onepiece.insert(5, 'month8', 0, True)
df_onepiece.insert(5, 'month7', 0, True)
df_onepiece.insert(5, 'month6', 0, True)
df_onepiece.insert(5, 'month5', 0, True)
df_onepiece.insert(5, 'month4', 0, True)
df_onepiece.insert(5, 'month3', 0, True)
df_onepiece.insert(5, 'month2', 0, True)
df_onepiece.insert(5, 'month1', 0, True)

In [442]:
df_onepiece.month1 = df_onepiece.apply(lambda x: 1 if x.dateMonth==1 else x.month1, axis=1)
df_onepiece.month2 = df_onepiece.apply(lambda x: 1 if x.dateMonth==2 else x.month2, axis=1)
df_onepiece.month3 = df_onepiece.apply(lambda x: 1 if x.dateMonth==3 else x.month3, axis=1)
df_onepiece.month4 = df_onepiece.apply(lambda x: 1 if x.dateMonth==4 else x.month4, axis=1)
df_onepiece.month5 = df_onepiece.apply(lambda x: 1 if x.dateMonth==5 else x.month5, axis=1)
df_onepiece.month6 = df_onepiece.apply(lambda x: 1 if x.dateMonth==6 else x.month6, axis=1)
df_onepiece.month7 = df_onepiece.apply(lambda x: 1 if x.dateMonth==7 else x.month7, axis=1)
df_onepiece.month8 = df_onepiece.apply(lambda x: 1 if x.dateMonth==8 else x.month8, axis=1)
df_onepiece.month9 = df_onepiece.apply(lambda x: 1 if x.dateMonth==9 else x.month9, axis=1)
df_onepiece.month10 = df_onepiece.apply(lambda x: 1 if x.dateMonth==10 else x.month10, axis=1)
df_onepiece.month11 = df_onepiece.apply(lambda x: 1 if x.dateMonth==11 else x.month11, axis=1)
df_onepiece.month12 = df_onepiece.apply(lambda x: 1 if x.dateMonth==12 else x.month12, axis=1)

In [443]:
from copy import deepcopy
a = deepcopy(df_onepiece)

In [444]:
k = a.groupby(['goodsNo'])['reviewContent'].apply(list).reset_index(name="reviewContents")
k

,goodsNo,reviewContents
0,230853,[[너무 귀엽고 상큼하게 입기 좋을거 같아요. 너무 귀여워요ㅠ]]
1,387218,[[생각보다 큰 사이즈로 가도 핏 되는 느낌이에요 참고!]]
2,435992,[[디자인이 독특해서 맘에 드네요 편하게 멋스럽게 잘 입을 듯 하네요 근데 구김 ...
3,653344,[[귀여운스타일이고레깅스와입으면데일리로좋아요]]
4,677397,[[편하고 예쁘게 입을 옷 찾고 있었는데 마음에 듭니다]]
...,...,...
1615,2386153,[[원단은 생각보다 얇아요! 그래서 여름에 덥지않게 입긴 좋을 듯해요 무난무난한 디...
1616,2396014,[[딱 맞고 예뻐요 기장도 넉넉하니 맘에 듭니다 잘 입을게요]]
1617,2396257,[[색상이 밝아서 블랙보다는 뚱뚱해보이긴 하지만 예뻐요~ 팔이 길게 나왔지만 크게 ...
1618,2396259,"[[예뻐요~ 카키색 같은 색상도 있으면 좋겠어요~ㅎㅎ 잘 입겠습니다!], [예뻐요~..."


In [445]:
k.reviewContents = k.reviewContents.astype('str')
k.reviewContents = k.apply(lambda x: x.reviewContents.replace("[", "").replace("]", ""), axis=1)

In [446]:
k.insert(2, 'reviewNoun', '', True)
k.reviewNoun = k.apply(lambda x: okt.nouns(x.reviewContents), axis=1)
k

,goodsNo,reviewContents,reviewNoun
0,230853,'너무 귀엽고 상큼하게 입기 좋을거 같아요. 너무 귀여워요ㅠ',[상큼]
1,387218,'생각보다 큰 사이즈로 가도 핏 되는 느낌이에요 참고!',"[생각, 사이즈, 가도, 핏, 느낌, 참고]"
2,435992,'디자인이 독특해서 맘에 드네요 편하게 멋스럽게 잘 입을 듯 하네요 근데 구김 잘...,"[디자인, 맘, 드네, 멋, 입, 듯, 구김, 원단]"
3,653344,'귀여운스타일이고레깅스와입으면데일리로좋아요',"[스타일, 레깅스, 일리]"
4,677397,'편하고 예쁘게 입을 옷 찾고 있었는데 마음에 듭니다',"[입, 옷, 마음, 듭니]"
...,...,...,...
1615,2386153,'원단은 생각보다 얇아요! 그래서 여름에 덥지않게 입긴 좋을 듯해요 무난무난한 디자...,"[원단, 생각, 여름, 듯, 무난, 무난, 디자인, 핏]"
1616,2396014,'딱 맞고 예뻐요 기장도 넉넉하니 맘에 듭니다 잘 입을게요',"[기장, 맘, 듭니]"
1617,2396257,'색상이 밝아서 블랙보다는 뚱뚱해보이긴 하지만 예뻐요~ 팔이 길게 나왔지만 크게 신...,"[색상, 블랙, 팔이, 크게, 신경, 정도, 색상, 블랙, 팔이, 크게, 신경, 정도]"
1618,2396259,"'예뻐요~ 카키색 같은 색상도 있으면 좋겠어요~ㅎㅎ 잘 입겠습니다!', '예뻐요~ ...","[카키색, 색상, 카키색, 색상]"


In [447]:
k = k.drop('reviewContents', axis=1)
k

,goodsNo,reviewNoun
0,230853,[상큼]
1,387218,"[생각, 사이즈, 가도, 핏, 느낌, 참고]"
2,435992,"[디자인, 맘, 드네, 멋, 입, 듯, 구김, 원단]"
3,653344,"[스타일, 레깅스, 일리]"
4,677397,"[입, 옷, 마음, 듭니]"
...,...,...
1615,2386153,"[원단, 생각, 여름, 듯, 무난, 무난, 디자인, 핏]"
1616,2396014,"[기장, 맘, 듭니]"
1617,2396257,"[색상, 블랙, 팔이, 크게, 신경, 정도, 색상, 블랙, 팔이, 크게, 신경, 정도]"
1618,2396259,"[카키색, 색상, 카키색, 색상]"


In [448]:
sub = pd.merge(left=sub, right=k, how="left", on='goodsNo')
sub

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun
0,1933622,none,19336222654,2654,"[다크, 그레이, 색상, 카키, 품, 매우, 커서, 사이즈, 구애, 수, 허리, 끈..."
1,2297184,M,22971842655,2655,"[길이, 감, 스타일, 느낌, 날, 보이지, 여유, 사이즈, 구매, 나그, 어깨, ..."
2,2297184,S,22971842656,2656,"[길이, 감, 스타일, 느낌, 날, 보이지, 여유, 사이즈, 구매, 나그, 어깨, ..."
3,2297184,XS,22971842657,2657,"[길이, 감, 스타일, 느낌, 날, 보이지, 여유, 사이즈, 구매, 나그, 어깨, ..."
4,1771393,MINT/FREE,17713932658,2658,"[일해, 구매, 가격, 정도, 것, 여름, 원피스, 맘, 가을, 원피스, 미리, 구..."
...,...,...,...,...,...
2042,1782233,M,17822334696,4696,"[절개, 디테, 일이, 마음, 핏라인, 색도, 봄, 컬러, 가격]"
2043,1790851,M,17908514697,4697,"[미디엄, 사이즈, 런가, 허리, 널럴, 옷]"
2044,1440679,NONE,14406794698,4698,"[패턴, 구매, 생각, 치마, 길이, 진짜, 치마, 길이]"
2045,1941295,PINK/FREE,19412954699,4699,"[원피스, 분, 정도, 밴딩, 다만, 난, 후, 겨드랑이, 부분, 붕떠, 잘못, 것..."


In [449]:
d = df_onepiece.groupby(['goodsNo', 'goodsSize']).sum()[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]
d

,,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
goodsNo,goodsSize,,,,,,,,,,,,
230853,none,0,0,0,0,0,0,1,0,0,0,0,0
387218,그레이헤더/S,0,0,0,0,0,0,0,1,0,0,0,0
435992,NONE,0,0,0,1,0,0,0,0,0,0,0,0
653344,none,0,0,0,0,0,0,0,0,0,0,1,0
677397,FREE,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2386153,M,0,0,1,0,0,0,0,0,0,0,0,0
2396014,M,0,0,1,0,0,0,0,0,0,0,0,0
2396257,FREE,0,0,2,0,0,0,0,0,0,0,0,0


#### 키, 몸무게 평균 구하기

In [450]:
c = df_onepiece.groupby(['goodsNo', 'goodsSize'], as_index=False).mean()[['goodsNo', 'goodsSize', 'userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]
c

,goodsNo,goodsSize,userHeight,userWeight,userSexMen,userSexWomen
0,230853,none,155.0,45.0,0.0,1.0
1,387218,그레이헤더/S,161.0,43.0,0.0,1.0
2,435992,NONE,162.0,55.0,0.0,1.0
3,653344,none,155.0,55.0,0.0,1.0
4,677397,FREE,163.0,55.0,0.0,1.0
...,...,...,...,...,...,...
2042,2386153,M,171.0,58.0,0.0,1.0
2043,2396014,M,173.0,53.0,0.0,1.0
2044,2396257,FREE,158.0,48.0,0.0,1.0
2045,2396259,FREE,158.0,48.0,0.0,1.0


In [451]:
sub.sort_values(['goodsNo', 'goodsSize'], inplace=True)
c.sort_values(['goodsNo', 'goodsSize'], inplace=True)
d.sort_values(['goodsNo', 'goodsSize'], inplace=True)
sub.index = range(len(d))
sub

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun
0,230853,none,23085304022,4022,[상큼]
1,387218,그레이헤더/S,38721803234,3234,"[생각, 사이즈, 가도, 핏, 느낌, 참고]"
2,435992,NONE,43599203203,3203,"[디자인, 맘, 드네, 멋, 입, 듯, 구김, 원단]"
3,653344,none,65334404183,4183,"[스타일, 레깅스, 일리]"
4,677397,FREE,67739703065,3065,"[입, 옷, 마음, 듭니]"
...,...,...,...,...,...
2042,2386153,M,23861533521,3521,"[원단, 생각, 여름, 듯, 무난, 무난, 디자인, 핏]"
2043,2396014,M,23960143328,3328,"[기장, 맘, 듭니]"
2044,2396257,FREE,23962573722,3722,"[색상, 블랙, 팔이, 크게, 신경, 정도, 색상, 블랙, 팔이, 크게, 신경, 정도]"
2045,2396259,FREE,23962593321,3321,"[카키색, 색상, 카키색, 색상]"


In [452]:
d.index = range(len(d))
d

,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2042,0,0,1,0,0,0,0,0,0,0,0,0
2043,0,0,1,0,0,0,0,0,0,0,0,0
2044,0,0,2,0,0,0,0,0,0,0,0,0
2045,0,0,2,0,0,0,0,0,0,0,0,0


In [453]:
sub = pd.concat([sub, c[['userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]], axis=1)
sub = pd.concat([sub, d[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]], axis=1)
sub.index = range(len(sub))
sub

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,230853,none,23085304022,4022,[상큼],155.0,45.0,0.0,1.0,0,0,0,0,0,0,1,0,0,0,0,0
1,387218,그레이헤더/S,38721803234,3234,"[생각, 사이즈, 가도, 핏, 느낌, 참고]",161.0,43.0,0.0,1.0,0,0,0,0,0,0,0,1,0,0,0,0
2,435992,NONE,43599203203,3203,"[디자인, 맘, 드네, 멋, 입, 듯, 구김, 원단]",162.0,55.0,0.0,1.0,0,0,0,1,0,0,0,0,0,0,0,0
3,653344,none,65334404183,4183,"[스타일, 레깅스, 일리]",155.0,55.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,1,0
4,677397,FREE,67739703065,3065,"[입, 옷, 마음, 듭니]",163.0,55.0,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2042,2386153,M,23861533521,3521,"[원단, 생각, 여름, 듯, 무난, 무난, 디자인, 핏]",171.0,58.0,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0
2043,2396014,M,23960143328,3328,"[기장, 맘, 듭니]",173.0,53.0,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0
2044,2396257,FREE,23962573722,3722,"[색상, 블랙, 팔이, 크게, 신경, 정도, 색상, 블랙, 팔이, 크게, 신경, 정도]",158.0,48.0,0.0,1.0,0,0,2,0,0,0,0,0,0,0,0,0
2045,2396259,FREE,23962593321,3321,"[카키색, 색상, 카키색, 색상]",158.0,48.0,0.0,1.0,0,0,2,0,0,0,0,0,0,0,0,0


In [454]:
df_onepiece = pd.merge(left=df_onepiece, right=sub[['goodsNo', 'goodsSize','newGoodsNo']], how="left", on=['goodsNo', 'goodsSize'])
df_onepiece.head()

,userName,dateYear,dateMonth,dateDay,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,odddr,2021,5,29,1933622,0,0,0,0,1,0,0,0,0,0,0,0,0,1,163,53,none,[다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 사...,https://image.msscdn.net/data/estimate/1933622...,1,1,2,2,0,0,0,0,0,19336222654
1,odddr,2021,5,29,1933622,0,0,0,0,1,0,0,0,0,0,0,0,0,1,163,53,none,"[여름에 편하게 입기 좋을것 같아요!, 생각보다 많이 파여서 조심해야ㅎㅐ요ㅠㅠ, 앞...",-,0,1,2,2,0,0,0,0,0,19336222654
2,뉴비_a7b53dc7,2022,2,25,2297184,0,1,0,0,0,0,0,0,0,0,0,0,0,1,160,55,M,"[길이감 좋고 스타일 꾸안꾸느낌, 빨리 날 좋아지면 입고싶음, 넉넉하게 나왔으나 퍼...",https://image.msscdn.net/data/estimate/2297184...,1,2,2,2,0,0,0,0,0,22971842655
3,뉴비_a7b53dc7,2022,2,25,2297184,0,1,0,0,0,0,0,0,0,0,0,0,0,1,160,55,M,"[여유있게 입으려고 M사이즈 구매, 나그랑이라 어깨 안넓어보이고 좋음, 트임이 생각...",-,0,2,2,2,0,0,0,0,0,22971842655
4,YnSUGA,2022,2,22,2297184,0,1,0,0,0,0,0,0,0,0,0,0,0,1,152,58,S,[봄 신상이라 그런가 확실히 얇아요. 키가 작은 편이라 기장 수선해서 입어야 할 것...,-,0,2,2,2,0,0,0,0,0,22971842656


In [455]:
len(sub['goodsNo'].unique())

1620

In [456]:
len(df_onepiece.newGoodsNo.unique())

2047

In [457]:
sub.shape

(2047, 21)

In [458]:
sub.to_csv('size_onepiece.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

In [459]:
sub.to_json('size_onepiece.json', orient='records', force_ascii=False)

### 그 외

In [650]:
df_onepiece['size'].value_counts()

2    6916
1    1103
3     428
0       1
Name: size, dtype: int64

In [651]:
df_onepiece.bright.value_counts()

2    6711
1     934
3     802
0       1
Name: bright, dtype: int64

In [652]:
df_onepiece.color.value_counts()

2    5667
1    2646
3     134
0       1
Name: color, dtype: int64

In [653]:
df_onepiece.thickness.value_counts()

0    8446
3       2
Name: thickness, dtype: int64

In [654]:
df_onepiece.weightness.value_counts()

0    8448
Name: weightness, dtype: int64

In [655]:
df_onepiece.touch.value_counts()

0    8448
Name: touch, dtype: int64

### helpNo, styleLikeNo는 모두 0이다. 제거 가능성 높아짐

In [656]:
df_onepiece.describe()

,dateYear,dateMonth,dateDay,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
count,8448.000000,8448.000000,8448.000000,8.448000e+03,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.0,8448.0,8448.0,8448.0,8.448000e+03
mean,2021.150331,6.627012,15.122514,1.813087e+06,0.079664,0.104167,0.081084,0.035985,0.059067,0.077770,0.109730,0.122633,0.089252,0.088778,0.072798,0.079072,0.017045,0.982955,162.436553,52.915601,0.250473,1.919863,1.984138,1.702415,0.000710,0.0,0.0,0.0,0.0,1.867238e+10
std,0.503225,3.419035,8.839225,3.012917e+05,0.270788,0.305495,0.272981,0.186263,0.235764,0.267825,0.312572,0.328034,0.285124,0.284440,0.259821,0.269867,0.129448,0.129448,5.925707,12.765438,0.433311,0.418687,0.453584,0.490935,0.046157,0.0,0.0,0.0,0.0,6.333033e+09
min,2020.000000,1.000000,1.000000,2.308530e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.007885e+10
25%,2021.000000,3.000000,7.000000,1.609158e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,159.000000,48.000000,0.000000,2.000000,2.000000,1.000000,0.000000,0.0,0.0,0.0,0.0,1.622447e+10
50%,2021.000000,7.000000,15.000000,1.877330e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,162.000000,52.000000,0.000000,2.000000,2.000000,2.000000,0.000000,0.0,0.0,0.0,0.0,1.888746e+10
75%,2021.000000,9.000000,23.000000,2.009663e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,166.000000,56.000000,1.000000,2.000000,2.000000,2.000000,0.000000,0.0,0.0,0.0,0.0,2.016787e+10
max,2022.000000,12.000000,31.000000,2.397414e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,200.000000,1000.000000,1.000000,3.000000,3.000000,3.000000,3.000000,0.0,0.0,0.0,0.0,9.930520e+10


In [978]:
df_onepiece.to_csv('review_onepiece.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

# Pants 리뷰 전처리

In [113]:
df_pants1 = pd.read_csv('./musinsa_review_pants_1.csv')
df_pants2 = pd.read_csv('./musinsa_review_pants_2.csv')

In [114]:
print(df_pants1.shape, df_pants2.shape)

(558110, 19) (946857, 19)


In [115]:
df_pants = pd.concat([df_pants1, df_pants2], axis=0)
df_pants.shape

(1504967, 19)

In [116]:
df_pants

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020.12.23,1281045,1,0,173cm,69kg,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946852,마산동호랭이,2020.08.10,1168907,1,0,178cm,84kg,34,"['핏 좋습니다 ', <br/>, '살짝 끼는 핏인데 소재가 좋아서 늘어날거같습니다']",NaN,0,2,2,2,2,0,0,0,0
946853,~TOP~,2020.08.10,1168907,1,0,174cm,79kg,32,['상품 잘 받았습니다 배송도 빠르게 잘 받았습니다'],NaN,0,2,2,2,2,0,0,0,0
946854,똘똘똘,2020.08.07,1168907,1,0,183cm,90kg,36,['가성비좋은 무탠다드 감사합니다 번창하세요 옷 괜찮습니다 감사합니다'],NaN,0,2,2,2,2,0,0,0,0
946855,브론즈_910d82e626b,2020.08.05,1168907,1,0,175cm,76kg,32,"['32인데 허리랑 허벅지가 좀 타이트해요', <br/>, '길이는 적당히 깁니다']",NaN,0,2,2,2,1,0,0,0,0


In [117]:
df_pants.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020.12.23,1281045,1,0,173cm,69kg,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0


In [118]:
df_pants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1504967 entries, 0 to 946856
Data columns (total 19 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   userName       1503404 non-null  object
 1   date           1504967 non-null  object
 2   goodsNo        1504967 non-null  int64 
 3   userSexMen     1504967 non-null  int64 
 4   userSexWomen   1504967 non-null  int64 
 5   userHeight     1504967 non-null  object
 6   userWeight     1504967 non-null  object
 7   goodsSize      1504967 non-null  object
 8   reviewContent  1504967 non-null  object
 9   reviewImg      434510 non-null   object
 10  reviewStyle    1504967 non-null  int64 
 11  size           1504967 non-null  int64 
 12  bright         1504967 non-null  int64 
 13  color          1504967 non-null  int64 
 14  thickness      1504967 non-null  int64 
 15  weightness     1504967 non-null  int64 
 16  touch          1504967 non-null  int64 
 17  helpNo         1504967 non-n

In [119]:
df_pants.shape

(1504967, 19)

### 탈퇴 인원 제거

In [120]:
df_pants.dropna(subset="userName", axis=0, inplace=True)
df_pants = df_pants.reset_index(drop=True)

In [121]:
df_pants.shape

(1503404, 19)

### Date를 년, 월, 일로 등분

In [122]:
df_pants.dropna(subset="date", axis=0, inplace=True)

In [123]:
df_pants.shape

(1503404, 19)

In [124]:
df_pants.insert(2, "dateDay", 0, True)
df_pants.insert(2, "dateMonth", 0, True)
df_pants.insert(2, "dateYear", 0, True)

In [125]:
df_pants.columns

Index(['userName', 'date', 'dateYear', 'dateMonth', 'dateDay', 'goodsNo',
       'userSexMen', 'userSexWomen', 'userHeight', 'userWeight', 'goodsSize',
       'reviewContent', 'reviewImg', 'reviewStyle', 'size', 'bright', 'color',
       'thickness', 'weightness', 'touch', 'helpNo', 'styleLikeNo'],
      dtype='object')

In [126]:
df_pants['dateYear'] = df_pants.apply(lambda x: x['date'].split('.')[0], axis=1)
df_pants['dateMonth'] = df_pants.apply(lambda x: x['date'].split('.')[1], axis=1)
df_pants['dateDay'] = df_pants.apply(lambda x: x['date'].split('.')[2], axis=1)

In [127]:
df_pants.head()

,userName,date,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022.01.10,2022,01,10,2258181,1,0,171cm,74kg,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021.12.24,2021,12,24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022.01.10,2022,01,10,2258181,1,0,171cm,74kg,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021.12.24,2021,12,24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020.12.23,2020,12,23,1281045,1,0,173cm,69kg,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0


In [128]:
df_pants = df_pants.drop('date', axis=1)

In [129]:
df_pants.dateYear = df_pants.dateYear.astype('int')
df_pants.dateMonth = df_pants.dateMonth.astype('int')
df_pants.dateDay = df_pants.dateDay.astype('int')

### goodsNo

In [130]:
df_pants.dropna(subset="goodsNo", axis=0, inplace=True)

In [131]:
df_pants.shape

(1503404, 21)

### 성별

In [132]:
df_pants[df_pants.userSexMen == 1].shape

(1289057, 21)

In [133]:
df_pants[df_pants.userSexWomen==1].shape

(214191, 21)

In [134]:
a = list((df_pants[df_pants.userSexMen==0][df_pants.userSexWomen==0]).index)
a

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12588\1405742730.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = list((df_pants[df_pants.userSexMen==0][df_pants.userSexWomen==0]).index)


[7171,
 18542,
 25500,
 36751,
 47062,
 71306,
 81862,
 99514,
 99554,
 111520,
 113794,
 118877,
 130966,
 132652,
 138269,
 160093,
 164777,
 183899,
 205717,
 205720,
 220326,
 224275,
 242825,
 252431,
 252905,
 252942,
 261982,
 262476,
 263052,
 294324,
 327051,
 331165,
 347768,
 347822,
 390972,
 428956,
 429008,
 441682,
 459069,
 463433,
 510674,
 510696,
 512850,
 512872,
 523450,
 523565,
 552769,
 556256,
 561145,
 565234,
 579109,
 586481,
 598967,
 598975,
 600264,
 615534,
 639162,
 644467,
 660585,
 661990,
 667649,
 684220,
 685896,
 685904,
 690435,
 693422,
 707263,
 707273,
 707505,
 726545,
 726553,
 748025,
 748027,
 748789,
 750169,
 758905,
 758909,
 770338,
 771845,
 772430,
 775729,
 781779,
 804721,
 815861,
 832935,
 848221,
 848222,
 848277,
 851855,
 851880,
 851919,
 851945,
 872400,
 872430,
 872889,
 887390,
 892911,
 897508,
 911937,
 911955,
 924481,
 951355,
 962372,
 969276,
 1033484,
 1061233,
 1068316,
 1075208,
 1088355,
 1093537,
 1093538,
 110

In [135]:
df_pants = df_pants.drop(a)
df_pants.reset_index(drop=True)

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022,1,10,2258181,1,0,171cm,74kg,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021,12,24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022,1,10,2258181,1,0,171cm,74kg,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021,12,24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020,12,23,1281045,1,0,173cm,69kg,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503243,마산동호랭이,2020,8,10,1168907,1,0,178cm,84kg,34,"['핏 좋습니다 ', <br/>, '살짝 끼는 핏인데 소재가 좋아서 늘어날거같습니다']",NaN,0,2,2,2,2,0,0,0,0
1503244,~TOP~,2020,8,10,1168907,1,0,174cm,79kg,32,['상품 잘 받았습니다 배송도 빠르게 잘 받았습니다'],NaN,0,2,2,2,2,0,0,0,0
1503245,똘똘똘,2020,8,7,1168907,1,0,183cm,90kg,36,['가성비좋은 무탠다드 감사합니다 번창하세요 옷 괜찮습니다 감사합니다'],NaN,0,2,2,2,2,0,0,0,0
1503246,브론즈_910d82e626b,2020,8,5,1168907,1,0,175cm,76kg,32,"['32인데 허리랑 허벅지가 좀 타이트해요', <br/>, '길이는 적당히 깁니다']",NaN,0,2,2,2,1,0,0,0,0


In [136]:
1289057+214191

1503248

### 키와 몸무게

In [137]:
df_pants.dropna(subset=["userHeight", "userWeight"], axis=0, inplace=True)

In [138]:
df_pants.shape

(1503248, 21)

In [139]:
df_pants.userHeight = df_pants.userHeight.apply(lambda x: x.replace("cm", "").replace(" ", ""))
df_pants.userHeight = df_pants.userHeight.astype('int64')

In [140]:
df_pants.userWeight = df_pants.userWeight.apply(lambda x: x.replace("kg", "").replace(" ", ""))
df_pants.userWeight = df_pants.userWeight.astype('int64')

In [141]:
a = list((df_pants[df_pants.userHeight==0][df_pants.userWeight==0]).index)
a

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12588\3922243988.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = list((df_pants[df_pants.userHeight==0][df_pants.userWeight==0]).index)


[1857,
 5977,
 41584,
 57173,
 93491,
 203467,
 208380,
 229242,
 263748,
 273395,
 327219,
 327529,
 354814,
 375034,
 523479,
 560404,
 562451,
 562452,
 611682,
 753052,
 787645,
 805291,
 861713,
 908501,
 908507,
 1061189,
 1136138,
 1143271,
 1179872,
 1258208,
 1381240,
 1404354,
 1408946,
 1431305,
 1440963,
 1467857]

In [142]:
df_pants = df_pants.drop(a)
df_pants.reset_index(drop=True)

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022,1,10,2258181,1,0,171,74,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021,12,24,2258181,1,0,175,80,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022,1,10,2258181,1,0,171,74,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021,12,24,2258181,1,0,175,80,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020,12,23,1281045,1,0,173,69,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503207,마산동호랭이,2020,8,10,1168907,1,0,178,84,34,"['핏 좋습니다 ', <br/>, '살짝 끼는 핏인데 소재가 좋아서 늘어날거같습니다']",NaN,0,2,2,2,2,0,0,0,0
1503208,~TOP~,2020,8,10,1168907,1,0,174,79,32,['상품 잘 받았습니다 배송도 빠르게 잘 받았습니다'],NaN,0,2,2,2,2,0,0,0,0
1503209,똘똘똘,2020,8,7,1168907,1,0,183,90,36,['가성비좋은 무탠다드 감사합니다 번창하세요 옷 괜찮습니다 감사합니다'],NaN,0,2,2,2,2,0,0,0,0
1503210,브론즈_910d82e626b,2020,8,5,1168907,1,0,175,76,32,"['32인데 허리랑 허벅지가 좀 타이트해요', <br/>, '길이는 적당히 깁니다']",NaN,0,2,2,2,1,0,0,0,0


### 이미지

In [143]:
df_pants.reviewStyle.value_counts()

0    1069286
1     433926
Name: reviewStyle, dtype: int64

In [144]:
df_pants.reviewImg = df_pants.reviewImg.fillna("-")

In [145]:
df_pants.reviewImg

0          https://image.msscdn.net/data/estimate/2258181...
1          https://image.msscdn.net/data/estimate/2258181...
2                                                          -
3                                                          -
4          https://image.msscdn.net/data/estimate/1281045...
                                 ...                        
1503399                                                    -
1503400                                                    -
1503401                                                    -
1503402                                                    -
1503403                                                    -
Name: reviewImg, Length: 1503212, dtype: object

### Contents

In [146]:
print(df_pants.reviewContent[0])
print(df_pants.reviewContent[1].split(","))

['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 그랬나바요', <br/>, '정말최고']
["['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱']"]


In [147]:
# span 태그가 들어가있는 리뷰내용 전처리

cnt = 0
result = []
for i in df_pants.reviewContent:
    if '<span>' in i:
        cnt += 1
        result.append(i)
print(cnt)

18


In [148]:
result[1]

'[\'저한테는 너무 크다는 느낌 없이 M사이즈 선택에 후회 없을 베스트 핏으로 잘 맞았어요! 색감도 쨍하고 원단도 탄탄해서 맘에 들어요. 자연광에 잘 어울리는 색감이고 요즘 날씨에 편하게 입기 좋네요. 민트도 사고 싶어졌어요ㅠ 그리고 은근히 뒤에 로고 포인트가 디자인의 킬포입니다 전 앞면 보고 반해서 샀는데 뒷판도 너무 예뻐요. 에어포스에 신으면 완벽합니다>\', \'!</div\', \'\\n\', <div class="review-evaluation">\r\n<ul class="review-evaluation__list">\r\n<li class="review-evaluation__item">사이즈 <span>보통이에요</span></li>\r\n<li class="review-evaluation__item">밝기 <span>밝아요</span></li>\r\n<li class="review-evaluation__item">색감 <span>선명해요</span></li>\r\n<li class="review-evaluation__item">두께감 <span>보통이에요</span></li>\r\n</ul>\r\n</div>, \'\\n\', \' 이미지 \', \'\\n\', <div class="review-content-photo">\r\n<div class="review-content-photo__wrap">\r\n<ul class="review-content-photo__list">\r\n<li class="review-content-photo__item" data-img-index="0" style="background-image:url(\'//image.msscdn.net/data/estimate/1554661_0/gallery_608c4885ca283.jpg.list\');">\r\n<img alt="아카이브 볼드(ARCHIVE BOLD) 939 LOGO SWEAT PANTS (DEEP RED) 후기" src="//image.msscdn.net/d

In [149]:
for i in result:
    idx = df_pants[df_pants.reviewContent==i].index[0]
    df_pants.loc[idx, 'reviewContent'] = i.split('</div')[0][:]

In [150]:
df_pants.reviewContent = df_pants.reviewContent.apply(lambda x: x.replace("[", "").replace("]", "").replace(" '", "").replace("' ", "").replace("'", "").replace(", <br/>", "").split(","))

In [151]:
print(df_pants.reviewContent[17889])
print(df_pants.reviewContent[1])

['너무 예뻐서 회색 검은색 두 컬러 구매 완료>.', ' 핏 좋고 편하고 따수워용']
['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱']


### 사이즈

In [152]:
sub = df_pants[["goodsNo", "goodsSize"]]
sub = sub.drop_duplicates(subset = ["goodsNo", "goodsSize"]).reset_index(drop=True)
sub.dropna(subset="goodsNo", inplace=True)
sub.insert(2, 'newGoodsNo', sub.goodsNo, True)
sub.insert(3, 'index', 0, True)
sub['index'] = range(4701, 4701+len(sub))
sub.index = range(4701, 4701+len(sub))
# sub.newGoodsNo = sub.newGoodsNo.apply(lambda x: round(x))

In [153]:
sub.goodsNo = sub.goodsNo.astype('str')
sub['index'] = sub['index'].astype('str')
sub.newGoodsNo = sub.newGoodsNo.astype('str')

In [154]:
len(sub)

74137

In [155]:
for i in range(4701, len(sub)+4701):
    if len(sub.newGoodsNo[i]) < 7:
        while len(sub.newGoodsNo[i]) < 7:
            sub.newGoodsNo[i] += '0'
sub

,goodsNo,goodsSize,newGoodsNo,index
4701,2258181,M,2258181,4701
4702,2258181,L,2258181,4702
4703,1281045,S,1281045,4703
4704,2328543,L,2328543,4704
4705,2346636,블랙/XL,2346636,4705
...,...,...,...,...
78833,1168907,33,1168907,78833
78834,1168907,36,1168907,78834
78835,1168907,38,1168907,78835
78836,1168907,40,1168907,78836


In [156]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74137 entries, 4701 to 78837
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   goodsNo     74137 non-null  object
 1   goodsSize   74137 non-null  object
 2   newGoodsNo  74137 non-null  object
 3   index       74137 non-null  object
dtypes: object(4)
memory usage: 2.3+ MB


In [157]:
sub['newGoodsNo'] = sub.apply(lambda x: x['newGoodsNo']+x['index'], axis=1)
sub

,goodsNo,goodsSize,newGoodsNo,index
4701,2258181,M,22581814701,4701
4702,2258181,L,22581814702,4702
4703,1281045,S,12810454703,4703
4704,2328543,L,23285434704,4704
4705,2346636,블랙/XL,23466364705,4705
...,...,...,...,...
78833,1168907,33,116890778833,78833
78834,1168907,36,116890778834,78834
78835,1168907,38,116890778835,78835
78836,1168907,40,116890778836,78836


In [158]:
sub.goodsNo = sub.goodsNo.astype('int')
sub.newGoodsNo = sub.newGoodsNo.astype('int64')

In [159]:
df_pants

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022,1,10,2258181,1,0,171,74,M,"[세트별로 다사서 이번엔 바지만 샀는데, 이것도 위에 같이 살걸 그랬나바요, 정말최고]",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021,12,24,2258181,1,0,175,80,L,[배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022,1,10,2258181,1,0,171,74,M,"[진짜 너무 이뻐서바지만 또사요, 정말 최고 아이템]",-,0,2,2,2,2,0,0,0,0
3,10407,2021,12,24,2258181,1,0,175,80,L,[배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱],-,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020,12,23,1281045,1,0,173,69,S,"[제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503399,마산동호랭이,2020,8,10,1168907,1,0,178,84,34,"[핏 좋습니다, 살짝 끼는 핏인데 소재가 좋아서 늘어날거같습니다]",-,0,2,2,2,2,0,0,0,0
1503400,~TOP~,2020,8,10,1168907,1,0,174,79,32,[상품 잘 받았습니다 배송도 빠르게 잘 받았습니다],-,0,2,2,2,2,0,0,0,0
1503401,똘똘똘,2020,8,7,1168907,1,0,183,90,36,[가성비좋은 무탠다드 감사합니다 번창하세요 옷 괜찮습니다 감사합니다],-,0,2,2,2,2,0,0,0,0
1503402,브론즈_910d82e626b,2020,8,5,1168907,1,0,175,76,32,"[32인데 허리랑 허벅지가 좀 타이트해요, 길이는 적당히 깁니다]",-,0,2,2,2,1,0,0,0,0


#### month 당 사는 옷 수

In [160]:
df_pants.insert(5, 'month12', 0, True)
df_pants.insert(5, 'month11', 0, True)
df_pants.insert(5, 'month10', 0, True)
df_pants.insert(5, 'month9', 0, True)
df_pants.insert(5, 'month8', 0, True)
df_pants.insert(5, 'month7', 0, True)
df_pants.insert(5, 'month6', 0, True)
df_pants.insert(5, 'month5', 0, True)
df_pants.insert(5, 'month4', 0, True)
df_pants.insert(5, 'month3', 0, True)
df_pants.insert(5, 'month2', 0, True)
df_pants.insert(5, 'month1', 0, True)

In [161]:
df_pants.month1 = df_pants.apply(lambda x: 1 if x.dateMonth==1 else x.month1, axis=1)
df_pants.month2 = df_pants.apply(lambda x: 1 if x.dateMonth==2 else x.month2, axis=1)
df_pants.month3 = df_pants.apply(lambda x: 1 if x.dateMonth==3 else x.month3, axis=1)
df_pants.month4 = df_pants.apply(lambda x: 1 if x.dateMonth==4 else x.month4, axis=1)
df_pants.month5 = df_pants.apply(lambda x: 1 if x.dateMonth==5 else x.month5, axis=1)
df_pants.month6 = df_pants.apply(lambda x: 1 if x.dateMonth==6 else x.month6, axis=1)
df_pants.month7 = df_pants.apply(lambda x: 1 if x.dateMonth==7 else x.month7, axis=1)
df_pants.month8 = df_pants.apply(lambda x: 1 if x.dateMonth==8 else x.month8, axis=1)
df_pants.month9 = df_pants.apply(lambda x: 1 if x.dateMonth==9 else x.month9, axis=1)
df_pants.month10 = df_pants.apply(lambda x: 1 if x.dateMonth==10 else x.month10, axis=1)
df_pants.month11 = df_pants.apply(lambda x: 1 if x.dateMonth==11 else x.month11, axis=1)
df_pants.month12 = df_pants.apply(lambda x: 1 if x.dateMonth==12 else x.month12, axis=1)

In [162]:
from copy import deepcopy
a = deepcopy(df_pants)

In [163]:
k = a.groupby(['goodsNo'])['reviewContent'].apply(list).reset_index(name="reviewContents")
k

,goodsNo,reviewContents
0,25868,[[약간 뻣뻣한 감이 있지만 아주 불편한 정도는 아니에요 생각보다 길었지만 롤업해서...
1,64576,[[튼튼하고 좋은데 좋네요 멀티캠이고 배송은 조금 느려서 아쉽..]]
2,64582,"[[질감은 전투복의 사각사각한 그 느낌입니다., 립스탑 원단이라서 많이 팽팽하지 않..."
3,67485,[[바지가 짱짱하다고 해야할까요 허벅지가 딱 조여지는느낌이어서 안늘어날거같고 핏이 ...
4,83107,"[[30입는데 m은 작습니다, 뻣뻣하지만 퀄은 좋아보입니다, 롤업을 해도 기장은 살..."
...,...,...
24854,2407517,"[[믿고 사는 노앙입니다., 베이지색 바지 찾다가, 노앙에 판매하는거 보고 바로 사..."
24855,2407719,"[[만족스럽게 잘 입고 다녀요! 지퍼를 잠궈도 예쁘고, 풀어도 생각보다는 괜찮네요!]]"
24856,2410780,[[예뻐요! 재질도 생각보다 되게 괜찮고 핏도 좋아요]]
24857,2411470,[[가볍게 입기 너무 좋고 핏도 적당하니 이뻐요 평소에 많이 입을 듯]]


In [164]:
k.reviewContents = k.reviewContents.astype('str')
k.reviewContents = k.apply(lambda x: x.reviewContents.replace("[", "").replace("]", ""), axis=1)

In [165]:
k.insert(2, 'reviewNoun1', '', True)
k.reviewNoun1 = k.apply(lambda x: Counter(okt.nouns(x.reviewContents)).most_common(100), axis=1)
k

,goodsNo,reviewContents,reviewNoun1
0,25868,'약간 뻣뻣한 감이 있지만 아주 불편한 정도는 아니에요 생각보다 길었지만 롤업해서 ...,"[(사이즈, 32), (핏, 29), (기장, 25), (바지, 20), (생, 1..."
1,64576,'튼튼하고 좋은데 좋네요 멀티캠이고 배송은 조금 느려서 아쉽..',"[(멀티, 1), (캠, 1), (배송, 1), (조금, 1)]"
2,64582,"'질감은 전투복의 사각사각한 그 느낌입니다.', '립스탑 원단이라서 많이 팽팽하지 ...","[(질감, 2), (전투복, 2), (그, 2), (느낌, 2), (립, 2), (..."
3,67485,'바지가 짱짱하다고 해야할까요 허벅지가 딱 조여지는느낌이어서 안늘어날거같고 핏이 살...,"[(핏, 34), (바지, 32), (사이즈, 27), (세탁, 15), (좀, 1..."
4,83107,"'30입는데 m은 작습니다', '뻣뻣하지만 퀄은 좋아보입니다', '롤업을 해도 기장...","[(기장, 2), (옷, 2), (안, 2), (은, 1), (퀄, 1), (롤업,..."
...,...,...,...
24854,2407517,"'믿고 사는 노앙입니다.', '베이지색 바지 찾다가', '노앙에 판매하는거 보고 바...","[(노, 1), (베이지색, 1), (바지, 1), (노앙, 1), (판매, 1),..."
24855,2407719,"'만족스럽게 잘 입고 다녀요! 지퍼를 잠궈도 예쁘고', ' 풀어도 생각보다는 괜찮네요!'","[(만족, 1), (지퍼, 1), (생각, 1)]"
24856,2410780,'예뻐요! 재질도 생각보다 되게 괜찮고 핏도 좋아요',"[(재질, 1), (생각, 1), (핏, 1)]"
24857,2411470,'가볍게 입기 너무 좋고 핏도 적당하니 이뻐요 평소에 많이 입을 듯',"[(핏, 1), (평소, 1), (입, 1), (듯, 1)]"


In [166]:
k.insert(3, 'reviewNoun', '', True)

In [167]:
for i in k.itertuples():
    kkk = []
    for x, y in k.reviewNoun1[i.Index]:
        kkkkk = {'text': x, 'value': y}
        kkk.append(kkkkk)
    k.at[i.Index, 'reviewNoun'] = kkk
k

,goodsNo,reviewContents,reviewNoun1,reviewNoun
0,25868,'약간 뻣뻣한 감이 있지만 아주 불편한 정도는 아니에요 생각보다 길었지만 롤업해서 ...,"[(사이즈, 32), (핏, 29), (기장, 25), (바지, 20), (생, 1...","[{'text': '사이즈', 'value': 32}, {'text': '핏', '..."
1,64576,'튼튼하고 좋은데 좋네요 멀티캠이고 배송은 조금 느려서 아쉽..',"[(멀티, 1), (캠, 1), (배송, 1), (조금, 1)]","[{'text': '멀티', 'value': 1}, {'text': '캠', 'va..."
2,64582,"'질감은 전투복의 사각사각한 그 느낌입니다.', '립스탑 원단이라서 많이 팽팽하지 ...","[(질감, 2), (전투복, 2), (그, 2), (느낌, 2), (립, 2), (...","[{'text': '질감', 'value': 2}, {'text': '전투복', '..."
3,67485,'바지가 짱짱하다고 해야할까요 허벅지가 딱 조여지는느낌이어서 안늘어날거같고 핏이 살...,"[(핏, 34), (바지, 32), (사이즈, 27), (세탁, 15), (좀, 1...","[{'text': '핏', 'value': 34}, {'text': '바지', 'v..."
4,83107,"'30입는데 m은 작습니다', '뻣뻣하지만 퀄은 좋아보입니다', '롤업을 해도 기장...","[(기장, 2), (옷, 2), (안, 2), (은, 1), (퀄, 1), (롤업,...","[{'text': '기장', 'value': 2}, {'text': '옷', 'va..."
...,...,...,...,...
24854,2407517,"'믿고 사는 노앙입니다.', '베이지색 바지 찾다가', '노앙에 판매하는거 보고 바...","[(노, 1), (베이지색, 1), (바지, 1), (노앙, 1), (판매, 1),...","[{'text': '노', 'value': 1}, {'text': '베이지색', '..."
24855,2407719,"'만족스럽게 잘 입고 다녀요! 지퍼를 잠궈도 예쁘고', ' 풀어도 생각보다는 괜찮네요!'","[(만족, 1), (지퍼, 1), (생각, 1)]","[{'text': '만족', 'value': 1}, {'text': '지퍼', 'v..."
24856,2410780,'예뻐요! 재질도 생각보다 되게 괜찮고 핏도 좋아요',"[(재질, 1), (생각, 1), (핏, 1)]","[{'text': '재질', 'value': 1}, {'text': '생각', 'v..."
24857,2411470,'가볍게 입기 너무 좋고 핏도 적당하니 이뻐요 평소에 많이 입을 듯',"[(핏, 1), (평소, 1), (입, 1), (듯, 1)]","[{'text': '핏', 'value': 1}, {'text': '평소', 'va..."


In [168]:
k = k.drop(['reviewContents', 'reviewNoun1'], axis=1)
k

,goodsNo,reviewNoun
0,25868,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '..."
1,64576,"[{'text': '멀티', 'value': 1}, {'text': '캠', 'va..."
2,64582,"[{'text': '질감', 'value': 2}, {'text': '전투복', '..."
3,67485,"[{'text': '핏', 'value': 34}, {'text': '바지', 'v..."
4,83107,"[{'text': '기장', 'value': 2}, {'text': '옷', 'va..."
...,...,...
24854,2407517,"[{'text': '노', 'value': 1}, {'text': '베이지색', '..."
24855,2407719,"[{'text': '만족', 'value': 1}, {'text': '지퍼', 'v..."
24856,2410780,"[{'text': '재질', 'value': 1}, {'text': '생각', 'v..."
24857,2411470,"[{'text': '핏', 'value': 1}, {'text': '평소', 'va..."


In [169]:
sub = pd.merge(left=sub, right=k, how="left", on='goodsNo')
sub

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun
0,2258181,M,22581814701,4701,"[{'text': '바지', 'value': 2}, {'text': '정말', 'v..."
1,2258181,L,22581814702,4702,"[{'text': '바지', 'value': 2}, {'text': '정말', 'v..."
2,1281045,S,12810454703,4703,"[{'text': '옷', 'value': 2}, {'text': '제', 'val..."
3,2328543,L,23285434704,4704,"[{'text': '다른', 'value': 1}, {'text': '스웨', 'v..."
4,2346636,블랙/XL,23466364705,4705,"[{'text': '생각', 'value': 1}, {'text': '기장', 'v..."
...,...,...,...,...,...
74132,1168907,33,116890778833,78833,"[{'text': '사이즈', 'value': 561}, {'text': '핏', ..."
74133,1168907,36,116890778834,78834,"[{'text': '사이즈', 'value': 561}, {'text': '핏', ..."
74134,1168907,38,116890778835,78835,"[{'text': '사이즈', 'value': 561}, {'text': '핏', ..."
74135,1168907,40,116890778836,78836,"[{'text': '사이즈', 'value': 561}, {'text': '핏', ..."


In [170]:
d = df_pants.groupby(['goodsNo', 'goodsSize']).sum()[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]
d

month1  month2  month3  month4  month5  month6  month7  \
goodsNo goodsSize                                                           
25868   28/28           0       0       1       1       0       0       0   
        28/30           0       0       0       0       0       0       1   
        28/32           0       0       0       0       0       0       0   
        29/28           0       0       2       0       1       0       1   
        29/30           0       0       0       0       0       0       0   
...                   ...     ...     ...     ...     ...     ...     ...   
2407517 30              0       0       1       0       0       0       0   
2407719 L               0       0       1       0       0       0       0   
2410780 L               0       0       1       0       0       0       0   
2411470 그레이/L(75)       0       0       1       0       0       0       0   
2413456 M(85)           0       0       2       0       0       0       0   

                   month8  month9  month10  month11  month12  
goodsNo goodsSize                                             
25868   28/28           0       1        2        1        0  
        28/30           0       0        0        0        0  
        28/32           0       1        0        0        0  
        29/28           0       1        2        1        1  
        29/30           1       0        1        0        0  
...                   ...     ...      ...      ...      ...  
2407517 30              0       0        0        0        0  
2407719 L               0       0        0        0        0  
2410780 L               0       0        0        0        0  
2411470 그레이/L(75)       0       0        0        0        0  
2413456 M(85)           0       0        0        0        0  

[74137 rows x 12 columns]

#### 키, 몸무게 평균 구하기

In [171]:
c = df_pants.groupby(['goodsNo', 'goodsSize'], as_index=False).mean()[['goodsNo', 'goodsSize', 'userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]
c

,goodsNo,goodsSize,userHeight,userWeight,userSexMen,userSexWomen
0,25868,28/28,167.833333,54.333333,0.666667,0.333333
1,25868,28/30,174.000000,53.000000,1.000000,0.000000
2,25868,28/32,167.000000,50.000000,0.000000,1.000000
3,25868,29/28,170.222222,63.444444,1.000000,0.000000
4,25868,29/30,174.500000,62.500000,1.000000,0.000000
...,...,...,...,...,...,...
74132,2407517,30,172.000000,65.000000,1.000000,0.000000
74133,2407719,L,183.000000,78.000000,1.000000,0.000000
74134,2410780,L,169.000000,56.000000,0.000000,1.000000
74135,2411470,그레이/L(75),167.000000,59.000000,0.000000,1.000000


In [172]:
sub.sort_values(['goodsNo', 'goodsSize'], inplace=True)
c.sort_values(['goodsNo', 'goodsSize'], inplace=True)
d.sort_values(['goodsNo', 'goodsSize'], inplace=True)
sub.index = range(len(d))
sub

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun
0,25868,28/28,258680052215,52215,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '..."
1,25868,28/30,258680052218,52218,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '..."
2,25868,28/32,258680052204,52204,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '..."
3,25868,29/28,258680052205,52205,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '..."
4,25868,29/30,258680052201,52201,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '..."
...,...,...,...,...,...
74132,2407517,30,240751752980,52980,"[{'text': '노', 'value': 1}, {'text': '베이지색', '..."
74133,2407719,L,240771913225,13225,"[{'text': '만족', 'value': 1}, {'text': '지퍼', 'v..."
74134,2410780,L,241078053057,53057,"[{'text': '재질', 'value': 1}, {'text': '생각', 'v..."
74135,2411470,그레이/L(75),241147046647,46647,"[{'text': '핏', 'value': 1}, {'text': '평소', 'va..."


In [173]:
d.index = range(len(d))
d

,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,0,0,1,1,0,0,0,0,1,2,1,0
1,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,2,0,1,0,1,0,1,2,1,1
4,0,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
74132,0,0,1,0,0,0,0,0,0,0,0,0
74133,0,0,1,0,0,0,0,0,0,0,0,0
74134,0,0,1,0,0,0,0,0,0,0,0,0
74135,0,0,1,0,0,0,0,0,0,0,0,0


In [174]:
sub = pd.concat([sub, c[['userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]], axis=1)
sub = pd.concat([sub, d[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]], axis=1)
sub.index = range(len(sub))
sub

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,25868,28/28,258680052215,52215,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",167.833333,54.333333,0.666667,0.333333,0,0,1,1,0,0,0,0,1,2,1,0
1,25868,28/30,258680052218,52218,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",174.000000,53.000000,1.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0
2,25868,28/32,258680052204,52204,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",167.000000,50.000000,0.000000,1.000000,0,0,0,0,0,0,0,0,1,0,0,0
3,25868,29/28,258680052205,52205,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",170.222222,63.444444,1.000000,0.000000,0,0,2,0,1,0,1,0,1,2,1,1
4,25868,29/30,258680052201,52201,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",174.500000,62.500000,1.000000,0.000000,0,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74132,2407517,30,240751752980,52980,"[{'text': '노', 'value': 1}, {'text': '베이지색', '...",172.000000,65.000000,1.000000,0.000000,0,0,1,0,0,0,0,0,0,0,0,0
74133,2407719,L,240771913225,13225,"[{'text': '만족', 'value': 1}, {'text': '지퍼', 'v...",183.000000,78.000000,1.000000,0.000000,0,0,1,0,0,0,0,0,0,0,0,0
74134,2410780,L,241078053057,53057,"[{'text': '재질', 'value': 1}, {'text': '생각', 'v...",169.000000,56.000000,0.000000,1.000000,0,0,1,0,0,0,0,0,0,0,0,0
74135,2411470,그레이/L(75),241147046647,46647,"[{'text': '핏', 'value': 1}, {'text': '평소', 'va...",167.000000,59.000000,0.000000,1.000000,0,0,1,0,0,0,0,0,0,0,0,0


In [175]:
sub.head()

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,25868,28/28,258680052215,52215,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",167.833333,54.333333,0.666667,0.333333,0,0,1,1,0,0,0,0,1,2,1,0
1,25868,28/30,258680052218,52218,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",174.000000,53.000000,1.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0
2,25868,28/32,258680052204,52204,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",167.000000,50.000000,0.000000,1.000000,0,0,0,0,0,0,0,0,1,0,0,0
3,25868,29/28,258680052205,52205,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",170.222222,63.444444,1.000000,0.000000,0,0,2,0,1,0,1,0,1,2,1,1
4,25868,29/30,258680052201,52201,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",174.500000,62.500000,1.000000,0.000000,0,0,0,0,0,0,0,1,0,1,0,0


In [176]:
df_pants = pd.merge(left=df_pants, right=sub[['goodsNo', 'goodsSize','newGoodsNo']], how="left", on=['goodsNo', 'goodsSize'])
df_pants.head()

,userName,dateYear,dateMonth,dateDay,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,기무태,2022,1,10,2258181,1,0,0,0,0,0,0,0,0,0,0,0,1,0,171,74,M,"[세트별로 다사서 이번엔 바지만 샀는데, 이것도 위에 같이 살걸 그랬나바요, 정말최고]",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0,22581814701
1,10407,2021,12,24,2258181,0,0,0,0,0,0,0,0,0,0,0,1,1,0,175,80,L,[배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0,22581814702
2,기무태,2022,1,10,2258181,1,0,0,0,0,0,0,0,0,0,0,0,1,0,171,74,M,"[진짜 너무 이뻐서바지만 또사요, 정말 최고 아이템]",-,0,2,2,2,2,0,0,0,0,22581814701
3,10407,2021,12,24,2258181,0,0,0,0,0,0,0,0,0,0,0,1,1,0,175,80,L,[배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱],-,0,2,2,2,2,0,0,0,0,22581814702
4,브론즈_f0b04c5803c,2020,12,23,1281045,0,0,0,0,0,0,0,0,0,0,0,1,1,0,173,69,S,"[제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0,12810454703


In [177]:
len(sub['goodsNo'].unique())

24859

In [178]:
sub.shape

(74137, 21)

In [179]:
sub.to_csv('size_pants.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

In [180]:
sub.to_json('size_pants.json', orient='records', force_ascii=False)

In [181]:
sub

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,25868,28/28,258680052215,52215,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",167.833333,54.333333,0.666667,0.333333,0,0,1,1,0,0,0,0,1,2,1,0
1,25868,28/30,258680052218,52218,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",174.000000,53.000000,1.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0
2,25868,28/32,258680052204,52204,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",167.000000,50.000000,0.000000,1.000000,0,0,0,0,0,0,0,0,1,0,0,0
3,25868,29/28,258680052205,52205,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",170.222222,63.444444,1.000000,0.000000,0,0,2,0,1,0,1,0,1,2,1,1
4,25868,29/30,258680052201,52201,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",174.500000,62.500000,1.000000,0.000000,0,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74132,2407517,30,240751752980,52980,"[{'text': '노', 'value': 1}, {'text': '베이지색', '...",172.000000,65.000000,1.000000,0.000000,0,0,1,0,0,0,0,0,0,0,0,0
74133,2407719,L,240771913225,13225,"[{'text': '만족', 'value': 1}, {'text': '지퍼', 'v...",183.000000,78.000000,1.000000,0.000000,0,0,1,0,0,0,0,0,0,0,0,0
74134,2410780,L,241078053057,53057,"[{'text': '재질', 'value': 1}, {'text': '생각', 'v...",169.000000,56.000000,0.000000,1.000000,0,0,1,0,0,0,0,0,0,0,0,0
74135,2411470,그레이/L(75),241147046647,46647,"[{'text': '핏', 'value': 1}, {'text': '평소', 'va...",167.000000,59.000000,0.000000,1.000000,0,0,1,0,0,0,0,0,0,0,0,0


### 그 외

In [182]:
df_pants['size'].value_counts()

2    1260258
1     196934
3      45966
0         54
Name: size, dtype: int64

In [183]:
df_pants.bright.value_counts()

2    1239560
1     148759
3     114839
0         54
Name: bright, dtype: int64

In [184]:
df_pants.color.value_counts()

2    1162858
1     323274
3      17026
0         54
Name: color, dtype: int64

In [185]:
df_pants.thickness.value_counts()

2    1214055
3     163610
1     123865
0       1682
Name: thickness, dtype: int64

In [186]:
df_pants.weightness.value_counts()

0    1503211
1          1
Name: weightness, dtype: int64

In [187]:
df_pants.touch.value_counts()

0    1503212
Name: touch, dtype: int64

### helpNo, styleLikeNo는 모두 0이다. 제거 가능성 높아짐

In [188]:
df_pants.describe()

,dateYear,dateMonth,dateDay,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
count,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1.503212e+06,1503212.0,1503212.0,1503212.0,1.503212e+06
mean,2.021061e+03,6.905231e+00,1.534927e+01,1.406315e+06,9.104970e-02,1.060755e-01,9.083882e-02,4.730071e-02,4.794733e-02,5.863977e-02,5.934160e-02,6.654950e-02,8.118150e-02,1.105719e-01,1.214479e-01,1.190557e-01,8.575198e-01,1.424802e-01,1.740676e+02,3.009606e+03,2.886659e-01,1.899498e+00,1.977363e+00,1.796199e+00,2.024202e+00,6.652422e-07,0.0,0.0,0.0,2.677105e+11
std,5.809992e-01,3.788964e+00,8.772099e+00,4.244288e+05,2.876799e-01,3.079344e-01,2.873799e-01,2.122814e-01,2.136549e-01,2.349493e-01,2.362630e-01,2.492402e-01,2.731137e-01,3.136014e-01,3.266472e-01,3.238542e-01,3.495421e-01,3.495421e-01,3.884454e+01,2.479095e+06,4.531424e-01,3.893975e-01,4.183155e-01,4.301057e-01,4.417359e-01,8.156238e-04,0.0,0.0,0.0,2.689880e+11
min,2.020000e+03,1.000000e+00,1.000000e+00,2.586800e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,1.003447e+10
25%,2.021000e+03,3.000000e+00,8.000000e+00,1.149329e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.700000e+02,6.200000e+01,0.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,0.000000e+00,0.0,0.0,0.0,1.278214e+11
50%,2.021000e+03,7.000000e+00,1.500000e+01,1.357771e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.750000e+02,7.000000e+01,0.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,0.000000e+00,0.0,0.0,0.0,1.591655e+11
75%,2.021000e+03,1.000000e+01,2.300000e+01,1.738192e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.790000e+02,7.500000e+01,1.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,0.000000e+00,0.0,0.0,0.0,2.067896e+11
max,2.022000e+03,1.200000e+01,3.100000e+01,2.413456e+06,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.555500e+04,2.147484e+09,1.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,1.000000e+00,0.0,0.0,0.0,9.999681e+11


In [976]:
df_pants.to_csv('review_pants.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

# Outer 리뷰 전처리

In [63]:
df_outer1 = pd.read_csv('./musinsa_review_outer_1.csv')
df_outer2 = pd.read_csv('./musinsa_review_outer_2.csv')

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3251: DtypeWarning: Columns (2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [64]:
print(df_outer1.shape, df_outer2.shape)

(530159, 19) (343280, 19)


In [65]:
df_outer = pd.concat([df_outer1, df_outer2], axis=0)
df_outer.shape

(873439, 19)

In [66]:
df_outer

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,뉴비_8e8c2e52077b,2021.09.18,1766979,1,0,180cm,61kg,L,"['제가 어두운 옷보단 밝은 옷을 좋아하는데 ', <br/>, '너무 밝지도 않고 ...",https://image.msscdn.net/data/estimate/1766979...,1,2,1,2,2,0.0,0.0,0.0,0.0
1,무신가신사,2022.03.15,2391602,0,1,165cm,46kg,FREE,['예뻐요 색상도 엄청 환해지고 상품이 봤을때 더 예뻐요'],NaN,0,2,2,2,2,0.0,2.0,0.0,0.0
2,뉴비_3c17ca57,2021.07.11,1962220,0,1,163cm,54kg,black/free,['재질이 보풀이 조금 생길거 같지만 부들부들 너무 좋네요 ㅠ 다른 색상도 구매 하...,NaN,0,2,2,2,2,0.0,2.0,0.0,0.0
3,푸쳐핸섭,2022.02.20,2252652,1,0,175cm,78kg,레드/L,"['빨간색 좋아해서 삿는뎅', <br/>, '맘에는 드는뎅', <br/>, '구명조...",NaN,0,2,2,1,1,0.0,0.0,0.0,0.0
4,mmd99,2021.12.10,1549580,1,0,178cm,90kg,FREE,['이뻐요 따듯해요 네이비 색상도 꼭 사고싶네요 굿굳!'],https://image.msscdn.net/data/estimate/1549580...,1,1,1,1,1,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343275,좡좡쓰,2020.12.20,1653581,0,1,162cm,50kg,M,['나름 만족하는데 털이 장난 아님. 돌돌이 한통은 다 써야 안빠질 듯여'],NaN,0,2,2,2,2,0.0,0.0,0.0,0.0
343276,좡좡쓰,2020.12.11,1653581,1,0,177cm,64kg,L,['이쁘고 따뜻한 건 좋은데 털빠짐이 엄청 납니다. 번거롭게 하네요.'],NaN,0,2,2,2,2,0.0,0.0,0.0,0.0
343277,연현흠,2020.12.04,1653581,1,0,173cm,65kg,L,"['한겨울에 입어도 될 정도로 뜨뜻~합니다', <br/>, '색감도 좋아요']",NaN,0,2,1,1,1,0.0,0.0,0.0,0.0
343278,뉴비_e8a414c8,2020.11.12,1653581,1,0,175cm,78kg,L,['두툼하고 생각보다 큽니다 겨울에 패딩대용으로도 가능할듯'],NaN,0,1,2,2,1,0.0,0.0,0.0,0.0


In [67]:
df_outer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 873439 entries, 0 to 343279
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   userName       872653 non-null  object 
 1   date           873439 non-null  object 
 2   goodsNo        873439 non-null  object 
 3   userSexMen     873439 non-null  object 
 4   userSexWomen   873439 non-null  object 
 5   userHeight     873438 non-null  object 
 6   userWeight     873439 non-null  object 
 7   goodsSize      873439 non-null  object 
 8   reviewContent  873439 non-null  object 
 9   reviewImg      245815 non-null  object 
 10  reviewStyle    873439 non-null  int64  
 11  size           873439 non-null  int64  
 12  bright         873439 non-null  int64  
 13  color          873439 non-null  int64  
 14  thickness      873439 non-null  int64  
 15  weightness     873438 non-null  float64
 16  touch          873438 non-null  float64
 17  helpNo         873438 non-nul

In [68]:
df_outer.shape

(873439, 19)

### 탈퇴 인원 제거

In [69]:
df_outer.dropna(subset=["userName"], axis=0, inplace=True)
df_outer = df_outer.reset_index(drop=True)

In [70]:
df_outer.shape

(872653, 19)

### Date를 년, 월, 일로 등분

In [71]:
df_outer.dropna(subset=["date"], axis=0, inplace=True)

In [72]:
df_outer.shape

(872653, 19)

In [73]:
df_outer.insert(2, "dateDay", 0, True)
df_outer.insert(2, "dateMonth", 0, True)
df_outer.insert(2, "dateYear", 0, True)

In [74]:
df_outer.columns

Index(['userName', 'date', 'dateYear', 'dateMonth', 'dateDay', 'goodsNo',
       'userSexMen', 'userSexWomen', 'userHeight', 'userWeight', 'goodsSize',
       'reviewContent', 'reviewImg', 'reviewStyle', 'size', 'bright', 'color',
       'thickness', 'weightness', 'touch', 'helpNo', 'styleLikeNo'],
      dtype='object')

In [75]:
result = []
for i in range(len(df_outer)):
    if len(df_outer.date[i].split('.')) != 3:
        result.append(i)
result

[67770, 528773]

In [76]:
df_outer = df_outer.drop(result)
df_outer.reset_index(drop=True)

,userName,date,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,뉴비_8e8c2e52077b,2021.09.18,0,0,0,1766979,1,0,180cm,61kg,L,"['제가 어두운 옷보단 밝은 옷을 좋아하는데 ', <br/>, '너무 밝지도 않고 ...",https://image.msscdn.net/data/estimate/1766979...,1,2,1,2,2,0.0,0.0,0.0,0.0
1,무신가신사,2022.03.15,0,0,0,2391602,0,1,165cm,46kg,FREE,['예뻐요 색상도 엄청 환해지고 상품이 봤을때 더 예뻐요'],NaN,0,2,2,2,2,0.0,2.0,0.0,0.0
2,뉴비_3c17ca57,2021.07.11,0,0,0,1962220,0,1,163cm,54kg,black/free,['재질이 보풀이 조금 생길거 같지만 부들부들 너무 좋네요 ㅠ 다른 색상도 구매 하...,NaN,0,2,2,2,2,0.0,2.0,0.0,0.0
3,푸쳐핸섭,2022.02.20,0,0,0,2252652,1,0,175cm,78kg,레드/L,"['빨간색 좋아해서 삿는뎅', <br/>, '맘에는 드는뎅', <br/>, '구명조...",NaN,0,2,2,1,1,0.0,0.0,0.0,0.0
4,mmd99,2021.12.10,0,0,0,1549580,1,0,178cm,90kg,FREE,['이뻐요 따듯해요 네이비 색상도 꼭 사고싶네요 굿굳!'],https://image.msscdn.net/data/estimate/1549580...,1,1,1,1,1,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872646,좡좡쓰,2020.12.20,0,0,0,1653581,0,1,162cm,50kg,M,['나름 만족하는데 털이 장난 아님. 돌돌이 한통은 다 써야 안빠질 듯여'],NaN,0,2,2,2,2,0.0,0.0,0.0,0.0
872647,좡좡쓰,2020.12.11,0,0,0,1653581,1,0,177cm,64kg,L,['이쁘고 따뜻한 건 좋은데 털빠짐이 엄청 납니다. 번거롭게 하네요.'],NaN,0,2,2,2,2,0.0,0.0,0.0,0.0
872648,연현흠,2020.12.04,0,0,0,1653581,1,0,173cm,65kg,L,"['한겨울에 입어도 될 정도로 뜨뜻~합니다', <br/>, '색감도 좋아요']",NaN,0,2,1,1,1,0.0,0.0,0.0,0.0
872649,뉴비_e8a414c8,2020.11.12,0,0,0,1653581,1,0,175cm,78kg,L,['두툼하고 생각보다 큽니다 겨울에 패딩대용으로도 가능할듯'],NaN,0,1,2,2,1,0.0,0.0,0.0,0.0


In [77]:
df_outer['dateYear'] = df_outer.apply(lambda x: x['date'].split('.')[0], axis=1)
df_outer['dateMonth'] = df_outer.apply(lambda x: x['date'].split('.')[1], axis=1)
df_outer['dateDay'] = df_outer.apply(lambda x: x['date'].split('.')[2], axis=1)

In [78]:
df_outer.shape

(872651, 22)

In [79]:
df_outer = df_outer.drop('date', axis=1)

In [80]:
df_outer.dateYear = df_outer.dateYear.astype('int')
df_outer.dateMonth = df_outer.dateMonth.astype('int')
df_outer.dateDay = df_outer.dateDay.astype('int')

### goodsNo

In [81]:
df_outer.dropna(subset=["goodsNo"], axis=0, inplace=True)

In [82]:
df_outer.shape

(872651, 21)

### 유저 성별

In [83]:
df_outer.userSexMen = df_outer.userSexMen.astype('int')
df_outer.userSexWomen = df_outer.userSexWomen.astype('int')

In [84]:
df_outer[df_outer.userSexMen == 1].shape

(658488, 21)

In [85]:
df_outer[df_outer.userSexWomen==1].shape

(214140, 21)

In [86]:
a = list((df_outer[(df_outer.userSexMen==0)&(df_outer.userSexWomen==0)]).index)
a

[111654,
 133210,
 133627,
 250427,
 304380,
 310991,
 321518,
 333749,
 333853,
 349904,
 394406,
 428030,
 438349,
 444273,
 444340,
 520274,
 758477,
 798887,
 799078,
 804272,
 812506,
 837884,
 858039]

In [87]:
df_outer = df_outer.drop(a)
df_outer.reset_index(drop=True)

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,뉴비_8e8c2e52077b,2021,9,18,1766979,1,0,180cm,61kg,L,"['제가 어두운 옷보단 밝은 옷을 좋아하는데 ', <br/>, '너무 밝지도 않고 ...",https://image.msscdn.net/data/estimate/1766979...,1,2,1,2,2,0.0,0.0,0.0,0.0
1,무신가신사,2022,3,15,2391602,0,1,165cm,46kg,FREE,['예뻐요 색상도 엄청 환해지고 상품이 봤을때 더 예뻐요'],NaN,0,2,2,2,2,0.0,2.0,0.0,0.0
2,뉴비_3c17ca57,2021,7,11,1962220,0,1,163cm,54kg,black/free,['재질이 보풀이 조금 생길거 같지만 부들부들 너무 좋네요 ㅠ 다른 색상도 구매 하...,NaN,0,2,2,2,2,0.0,2.0,0.0,0.0
3,푸쳐핸섭,2022,2,20,2252652,1,0,175cm,78kg,레드/L,"['빨간색 좋아해서 삿는뎅', <br/>, '맘에는 드는뎅', <br/>, '구명조...",NaN,0,2,2,1,1,0.0,0.0,0.0,0.0
4,mmd99,2021,12,10,1549580,1,0,178cm,90kg,FREE,['이뻐요 따듯해요 네이비 색상도 꼭 사고싶네요 굿굳!'],https://image.msscdn.net/data/estimate/1549580...,1,1,1,1,1,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872623,좡좡쓰,2020,12,20,1653581,0,1,162cm,50kg,M,['나름 만족하는데 털이 장난 아님. 돌돌이 한통은 다 써야 안빠질 듯여'],NaN,0,2,2,2,2,0.0,0.0,0.0,0.0
872624,좡좡쓰,2020,12,11,1653581,1,0,177cm,64kg,L,['이쁘고 따뜻한 건 좋은데 털빠짐이 엄청 납니다. 번거롭게 하네요.'],NaN,0,2,2,2,2,0.0,0.0,0.0,0.0
872625,연현흠,2020,12,4,1653581,1,0,173cm,65kg,L,"['한겨울에 입어도 될 정도로 뜨뜻~합니다', <br/>, '색감도 좋아요']",NaN,0,2,1,1,1,0.0,0.0,0.0,0.0
872626,뉴비_e8a414c8,2020,11,12,1653581,1,0,175cm,78kg,L,['두툼하고 생각보다 큽니다 겨울에 패딩대용으로도 가능할듯'],NaN,0,1,2,2,1,0.0,0.0,0.0,0.0


In [88]:
df_outer.shape

(872628, 21)

In [89]:
1289057+214191

1503248

### 키와 몸무게

In [90]:
df_outer.dropna(subset=["userHeight", "userWeight"], axis=0, inplace=True)

In [91]:
df_outer.shape

(872628, 21)

In [92]:
df_outer[df_outer.userHeight=='<br/>']

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo


In [93]:
df_outer.userHeight = df_outer.userHeight.apply(lambda x: x.replace("cm", "").replace(" ", ""))
df_outer.userHeight = df_outer.userHeight.astype('int64')

In [94]:
df_outer.userWeight = df_outer.userWeight.apply(lambda x: x.replace("kg", "").replace(" ", ""))
df_outer.userWeight = df_outer.userWeight.astype('int64')

In [95]:
a = list((df_outer[(df_outer.userWeight==0)&(df_outer.userHeight==0)]).index)
a

[16530,
 205413,
 245163,
 325996,
 328326,
 466564,
 509047,
 509334,
 577679,
 628400,
 765167,
 768593,
 805940,
 856238,
 865576,
 865577,
 865581]

In [96]:
df_outer = df_outer.drop(a)
df_outer.reset_index(drop=True)

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,뉴비_8e8c2e52077b,2021,9,18,1766979,1,0,180,61,L,"['제가 어두운 옷보단 밝은 옷을 좋아하는데 ', <br/>, '너무 밝지도 않고 ...",https://image.msscdn.net/data/estimate/1766979...,1,2,1,2,2,0.0,0.0,0.0,0.0
1,무신가신사,2022,3,15,2391602,0,1,165,46,FREE,['예뻐요 색상도 엄청 환해지고 상품이 봤을때 더 예뻐요'],NaN,0,2,2,2,2,0.0,2.0,0.0,0.0
2,뉴비_3c17ca57,2021,7,11,1962220,0,1,163,54,black/free,['재질이 보풀이 조금 생길거 같지만 부들부들 너무 좋네요 ㅠ 다른 색상도 구매 하...,NaN,0,2,2,2,2,0.0,2.0,0.0,0.0
3,푸쳐핸섭,2022,2,20,2252652,1,0,175,78,레드/L,"['빨간색 좋아해서 삿는뎅', <br/>, '맘에는 드는뎅', <br/>, '구명조...",NaN,0,2,2,1,1,0.0,0.0,0.0,0.0
4,mmd99,2021,12,10,1549580,1,0,178,90,FREE,['이뻐요 따듯해요 네이비 색상도 꼭 사고싶네요 굿굳!'],https://image.msscdn.net/data/estimate/1549580...,1,1,1,1,1,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872606,좡좡쓰,2020,12,20,1653581,0,1,162,50,M,['나름 만족하는데 털이 장난 아님. 돌돌이 한통은 다 써야 안빠질 듯여'],NaN,0,2,2,2,2,0.0,0.0,0.0,0.0
872607,좡좡쓰,2020,12,11,1653581,1,0,177,64,L,['이쁘고 따뜻한 건 좋은데 털빠짐이 엄청 납니다. 번거롭게 하네요.'],NaN,0,2,2,2,2,0.0,0.0,0.0,0.0
872608,연현흠,2020,12,4,1653581,1,0,173,65,L,"['한겨울에 입어도 될 정도로 뜨뜻~합니다', <br/>, '색감도 좋아요']",NaN,0,2,1,1,1,0.0,0.0,0.0,0.0
872609,뉴비_e8a414c8,2020,11,12,1653581,1,0,175,78,L,['두툼하고 생각보다 큽니다 겨울에 패딩대용으로도 가능할듯'],NaN,0,1,2,2,1,0.0,0.0,0.0,0.0


In [97]:
df_outer.shape

(872611, 21)

### 이미지

In [98]:
df_outer.reviewStyle.value_counts()

0    627018
1    245593
Name: reviewStyle, dtype: int64

In [99]:
df_outer.reviewImg = df_outer.reviewImg.fillna("-")

In [100]:
df_outer.reviewImg

0         https://image.msscdn.net/data/estimate/1766979...
1                                                         -
2                                                         -
3                                                         -
4         https://image.msscdn.net/data/estimate/1549580...
                                ...                        
872648                                                    -
872649                                                    -
872650                                                    -
872651                                                    -
872652                                                    -
Name: reviewImg, Length: 872611, dtype: object

### Contents

In [101]:
print(df_outer.reviewContent[0])
print(df_outer.reviewContent[1].split(","))

['제가 어두운 옷보단 밝은 옷을 좋아하는데 ', <br/>, '너무 밝지도 않고 딱 적당합니다.', <br/>, <br/>, '안 두꺼운데 얇은 편이라 가을과 봄에 입으면 딱 좋고', <br/>, <br/>, '교복같은 무슨 옷이던 잘 어울려서 더 좋은거 같아요.', <br/>, <br/>, '디자인이 과하고 좀 튈거라 생각했는데 ', <br/>, '과하지 않고 깔끔해요.', <br/>, <br/>, '단점은 핏이 조금 깔끔하지 못할때도 있어요.']
["['예뻐요 색상도 엄청 환해지고 상품이 봤을때 더 예뻐요']"]


In [102]:
# span 태그가 들어가있는 리뷰내용 전처리

cnt = 0
result = []
for i in df_outer.reviewContent:
    if '<span>' in i:
        cnt += 1
        result.append(i)
print(cnt)

21


In [103]:
for i in result:
    idx = df_outer[df_outer.reviewContent==i].index[0]
    df_outer.loc[idx, 'reviewContent'] = i.split('</div')[0][:]

In [104]:
df_outer.reviewContent = df_outer.reviewContent.apply(lambda x: x.replace("[", "").replace("]", "").replace(" '", "").replace("'", "").replace(", <br/>", "").split(","))

In [105]:
print(df_outer.reviewContent[0])
print(df_outer.reviewContent[1])

['제가 어두운 옷보단 밝은 옷을 좋아하는데', '너무 밝지도 않고 딱 적당합니다.', '안 두꺼운데 얇은 편이라 가을과 봄에 입으면 딱 좋고', '교복같은 무슨 옷이던 잘 어울려서 더 좋은거 같아요.', '디자인이 과하고 좀 튈거라 생각했는데', '과하지 않고 깔끔해요.', '단점은 핏이 조금 깔끔하지 못할때도 있어요.']
['예뻐요 색상도 엄청 환해지고 상품이 봤을때 더 예뻐요']


### 사이즈

In [107]:
sub = df_outer[["goodsNo", "goodsSize"]]
sub = sub.drop_duplicates(subset = ["goodsNo", "goodsSize"]).reset_index(drop=True)
sub.dropna(subset=["goodsNo"], inplace=True)
sub.insert(2, 'newGoodsNo', sub.goodsNo, True)
sub.insert(3, 'index', 0, True)
sub['index'] = range(78839, 78839+len(sub))
sub.index = range(78839, 78839+len(sub))
# sub.newGoodsNo = sub.newGoodsNo.apply(lambda x: round(x))

In [108]:
sub.goodsNo = sub.goodsNo.astype('str')
sub['index'] = sub['index'].astype('str')
sub.newGoodsNo = sub.newGoodsNo.astype('str')

In [109]:
len(sub)

52546

In [110]:
for i in range(78839, len(sub)+78839):
    if len(sub.newGoodsNo[i]) < 7:
        while len(sub.newGoodsNo[i]) < 7:
            sub.newGoodsNo[i] += '0'
sub

,goodsNo,goodsSize,newGoodsNo,index
78839,1766979,L,1766979,78839
78840,2391602,FREE,2391602,78840
78841,1962220,black/free,1962220,78841
78842,2252652,레드/L,2252652,78842
78843,1549580,FREE,1549580,78843
...,...,...,...,...
131380,1229923,XL,1229923,131380
131381,2107087,FREE,2107087,131381
131382,1653581,M,1653581,131382
131383,1653581,L,1653581,131383


In [111]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52546 entries, 78839 to 131384
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   goodsNo     52546 non-null  object
 1   goodsSize   52546 non-null  object
 2   newGoodsNo  52546 non-null  object
 3   index       52546 non-null  object
dtypes: object(4)
memory usage: 1.6+ MB


In [112]:
sub['newGoodsNo'] = sub.apply(lambda x: x['newGoodsNo']+x['index'], axis=1)
sub

,goodsNo,goodsSize,newGoodsNo,index
78839,1766979,L,176697978839,78839
78840,2391602,FREE,239160278840,78840
78841,1962220,black/free,196222078841,78841
78842,2252652,레드/L,225265278842,78842
78843,1549580,FREE,154958078843,78843
...,...,...,...,...
131380,1229923,XL,1229923131380,131380
131381,2107087,FREE,2107087131381,131381
131382,1653581,M,1653581131382,131382
131383,1653581,L,1653581131383,131383


In [113]:
sub.goodsNo = sub.goodsNo.astype('int')
sub.newGoodsNo = sub.newGoodsNo.astype('int64')

In [114]:
df_outer

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,뉴비_8e8c2e52077b,2021,9,18,1766979,1,0,180,61,L,"[제가 어두운 옷보단 밝은 옷을 좋아하는데, 너무 밝지도 않고 딱 적당합니다., 안...",https://image.msscdn.net/data/estimate/1766979...,1,2,1,2,2,0.0,0.0,0.0,0.0
1,무신가신사,2022,3,15,2391602,0,1,165,46,FREE,[예뻐요 색상도 엄청 환해지고 상품이 봤을때 더 예뻐요],-,0,2,2,2,2,0.0,2.0,0.0,0.0
2,뉴비_3c17ca57,2021,7,11,1962220,0,1,163,54,black/free,[재질이 보풀이 조금 생길거 같지만 부들부들 너무 좋네요 ㅠ 다른 색상도 구매 하려구요],-,0,2,2,2,2,0.0,2.0,0.0,0.0
3,푸쳐핸섭,2022,2,20,2252652,1,0,175,78,레드/L,"[빨간색 좋아해서 삿는뎅, 맘에는 드는뎅, 구명조끼 같다는 동료의 말에 빵터짐, 필...",-,0,2,2,1,1,0.0,0.0,0.0,0.0
4,mmd99,2021,12,10,1549580,1,0,178,90,FREE,[이뻐요 따듯해요 네이비 색상도 꼭 사고싶네요 굿굳!],https://image.msscdn.net/data/estimate/1549580...,1,1,1,1,1,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872648,좡좡쓰,2020,12,20,1653581,0,1,162,50,M,[나름 만족하는데 털이 장난 아님. 돌돌이 한통은 다 써야 안빠질 듯여],-,0,2,2,2,2,0.0,0.0,0.0,0.0
872649,좡좡쓰,2020,12,11,1653581,1,0,177,64,L,[이쁘고 따뜻한 건 좋은데 털빠짐이 엄청 납니다. 번거롭게 하네요.],-,0,2,2,2,2,0.0,0.0,0.0,0.0
872650,연현흠,2020,12,4,1653581,1,0,173,65,L,"[한겨울에 입어도 될 정도로 뜨뜻~합니다, 색감도 좋아요]",-,0,2,1,1,1,0.0,0.0,0.0,0.0
872651,뉴비_e8a414c8,2020,11,12,1653581,1,0,175,78,L,[두툼하고 생각보다 큽니다 겨울에 패딩대용으로도 가능할듯],-,0,1,2,2,1,0.0,0.0,0.0,0.0


#### month 당 사는 옷 수

In [317]:
df_outer.insert(5, 'month12', 0, True)
df_outer.insert(5, 'month11', 0, True)
df_outer.insert(5, 'month10', 0, True)
df_outer.insert(5, 'month9', 0, True)
df_outer.insert(5, 'month8', 0, True)
df_outer.insert(5, 'month7', 0, True)
df_outer.insert(5, 'month6', 0, True)
df_outer.insert(5, 'month5', 0, True)
df_outer.insert(5, 'month4', 0, True)
df_outer.insert(5, 'month3', 0, True)
df_outer.insert(5, 'month2', 0, True)
df_outer.insert(5, 'month1', 0, True)

In [318]:
df_outer.month1 = df_outer.apply(lambda x: 1 if x.dateMonth==1 else x.month1, axis=1)
df_outer.month2 = df_outer.apply(lambda x: 1 if x.dateMonth==2 else x.month2, axis=1)
df_outer.month3 = df_outer.apply(lambda x: 1 if x.dateMonth==3 else x.month3, axis=1)
df_outer.month4 = df_outer.apply(lambda x: 1 if x.dateMonth==4 else x.month4, axis=1)
df_outer.month5 = df_outer.apply(lambda x: 1 if x.dateMonth==5 else x.month5, axis=1)
df_outer.month6 = df_outer.apply(lambda x: 1 if x.dateMonth==6 else x.month6, axis=1)
df_outer.month7 = df_outer.apply(lambda x: 1 if x.dateMonth==7 else x.month7, axis=1)
df_outer.month8 = df_outer.apply(lambda x: 1 if x.dateMonth==8 else x.month8, axis=1)
df_outer.month9 = df_outer.apply(lambda x: 1 if x.dateMonth==9 else x.month9, axis=1)
df_outer.month10 = df_outer.apply(lambda x: 1 if x.dateMonth==10 else x.month10, axis=1)
df_outer.month11 = df_outer.apply(lambda x: 1 if x.dateMonth==11 else x.month11, axis=1)
df_outer.month12 = df_outer.apply(lambda x: 1 if x.dateMonth==12 else x.month12, axis=1)

In [319]:
from copy import deepcopy
a = deepcopy(df_outer)

In [320]:
k = a.groupby(['goodsNo'])['reviewContent'].apply(list).reset_index(name="reviewContents")
k

,goodsNo,reviewContents
0,18798,[[몸에 딱 맞는 모양의 후디입니다. 박시한 코디 원하시는 분 한테는 맞지 않습니다...
1,50937,"[[이쁘고 좀 더운데 가을에 입기 좋으 것 같아요, 사이즌 정사이즈에요], [실물로..."
2,55782,"[[좋습니다 좋은 가격에 좋은상품 잘 산것습니다 감사합니다 ~^^], [일단 조금큰..."
3,55892,"[[얇지 않고 누빔도 있어서 따뜻해요., 가죽 퀄리티도 좋아요., 한겨울에 입기에도..."
4,56669,"[[기모가 들어있어 따뜻하고 사이즈도 적당하네요., 아웃터로 입다가 더 추워지면 패..."
...,...,...
23550,978152,[[가격도 괜찮고 제질도 좋네여 대만족입니다!! 짱짱!!]]
23551,981292,[[제일 기대했던 상품인 만큼 좋게 나와서 기쁘네요. 밝은 계열의 옷을 도전해봤는데...
23552,981585,"[[사이즈 좋고 너무 이뻐여 또 사고싶어여 완전 강추], [옷은 저스트 핏입니다. ..."
23553,987007,[[좋아요좋아요좋아요좋아요 좋아요 좋아요좋아요좋아요좋아요]]


In [321]:
k.reviewContents = k.reviewContents.astype('str')
k.reviewContents = k.apply(lambda x: x.reviewContents.replace("[", "").replace("]", ""), axis=1)

In [322]:
k.insert(2, 'reviewNoun1', '', True)
k.reviewNoun1 = k.apply(lambda x: Counter(okt.nouns(x.reviewContents)).most_common(100), axis=1)
k

,goodsNo,reviewContents,reviewNoun1
0,18798,'몸에 딱 맞는 모양의 후디입니다. 박시한 코디 원하시는 분 한테는 맞지 않습니다....,"[(몸, 1), (모양, 1), (후디, 1), (박, 1), (시한, 1), (코..."
1,50937,"'이쁘고 좀 더운데 가을에 입기 좋으 것 같아요', '사이즌 정사이즈에요', '실물...","[(가격, 2), (옷, 2), (좀, 1), (가을, 1), (것, 1), (즌,..."
2,55782,"'좋습니다 좋은 가격에 좋은상품 잘 산것습니다 감사합니다 ~^^', '일단 조금큰거...","[(옷, 2), (가격, 1), (상품, 1), (일단, 1), (조금, 1), (..."
3,55892,"'얇지 않고 누빔도 있어서 따뜻해요.', '가죽 퀄리티도 좋아요.', '한겨울에 입...","[(사이즈, 11), (핏, 10), (만족, 8), (가격, 7), (버핏, 6)..."
4,56669,"'기모가 들어있어 따뜻하고 사이즈도 적당하네요.', '아웃터로 입다가 더 추워지면 ...","[(것, 10), (사이즈, 9), (기모, 5), (더, 5), (두께, 5), ..."
...,...,...,...
23550,978152,'가격도 괜찮고 제질도 좋네여 대만족입니다!! 짱짱!!',"[(가격, 1), (질도, 1), (여, 1), (만족, 1), (짱짱, 1)]"
23551,981292,'제일 기대했던 상품인 만큼 좋게 나와서 기쁘네요. 밝은 계열의 옷을 도전해봤는데 ...,"[(제일, 1), (상품, 1), (만큼, 1), (계열, 1), (옷, 1), (..."
23552,981585,"'사이즈 좋고 너무 이뻐여 또 사고싶어여 완전 강추', '옷은 저스트 핏입니다. 대...","[(구매, 2), (사이즈, 1), (또, 1), (완전, 1), (강추, 1), ..."
23553,987007,'좋아요좋아요좋아요좋아요 좋아요 좋아요좋아요좋아요좋아요',[]


In [323]:
k.insert(3, 'reviewNoun', '', True)

In [324]:
for i in k.itertuples():
    kkk = []
    for x, y in k.reviewNoun1[i.Index]:
        kkkkk = {'text': x, 'value': y}
        kkk.append(kkkkk)
    k.at[i.Index, 'reviewNoun'] = kkk
k

,goodsNo,reviewContents,reviewNoun1,reviewNoun
0,18798,'몸에 딱 맞는 모양의 후디입니다. 박시한 코디 원하시는 분 한테는 맞지 않습니다....,"[(몸, 1), (모양, 1), (후디, 1), (박, 1), (시한, 1), (코...","[{'text': '몸', 'value': 1}, {'text': '모양', 'va..."
1,50937,"'이쁘고 좀 더운데 가을에 입기 좋으 것 같아요', '사이즌 정사이즈에요', '실물...","[(가격, 2), (옷, 2), (좀, 1), (가을, 1), (것, 1), (즌,...","[{'text': '가격', 'value': 2}, {'text': '옷', 'va..."
2,55782,"'좋습니다 좋은 가격에 좋은상품 잘 산것습니다 감사합니다 ~^^', '일단 조금큰거...","[(옷, 2), (가격, 1), (상품, 1), (일단, 1), (조금, 1), (...","[{'text': '옷', 'value': 2}, {'text': '가격', 'va..."
3,55892,"'얇지 않고 누빔도 있어서 따뜻해요.', '가죽 퀄리티도 좋아요.', '한겨울에 입...","[(사이즈, 11), (핏, 10), (만족, 8), (가격, 7), (버핏, 6)...","[{'text': '사이즈', 'value': 11}, {'text': '핏', '..."
4,56669,"'기모가 들어있어 따뜻하고 사이즈도 적당하네요.', '아웃터로 입다가 더 추워지면 ...","[(것, 10), (사이즈, 9), (기모, 5), (더, 5), (두께, 5), ...","[{'text': '것', 'value': 10}, {'text': '사이즈', '..."
...,...,...,...,...
23550,978152,'가격도 괜찮고 제질도 좋네여 대만족입니다!! 짱짱!!',"[(가격, 1), (질도, 1), (여, 1), (만족, 1), (짱짱, 1)]","[{'text': '가격', 'value': 1}, {'text': '질도', 'v..."
23551,981292,'제일 기대했던 상품인 만큼 좋게 나와서 기쁘네요. 밝은 계열의 옷을 도전해봤는데 ...,"[(제일, 1), (상품, 1), (만큼, 1), (계열, 1), (옷, 1), (...","[{'text': '제일', 'value': 1}, {'text': '상품', 'v..."
23552,981585,"'사이즈 좋고 너무 이뻐여 또 사고싶어여 완전 강추', '옷은 저스트 핏입니다. 대...","[(구매, 2), (사이즈, 1), (또, 1), (완전, 1), (강추, 1), ...","[{'text': '구매', 'value': 2}, {'text': '사이즈', '..."
23553,987007,'좋아요좋아요좋아요좋아요 좋아요 좋아요좋아요좋아요좋아요',[],[]


In [325]:
k = k.drop(['reviewContents', 'reviewNoun1'], axis=1)
k

,goodsNo,reviewNoun
0,18798,"[{'text': '몸', 'value': 1}, {'text': '모양', 'va..."
1,50937,"[{'text': '가격', 'value': 2}, {'text': '옷', 'va..."
2,55782,"[{'text': '옷', 'value': 2}, {'text': '가격', 'va..."
3,55892,"[{'text': '사이즈', 'value': 11}, {'text': '핏', '..."
4,56669,"[{'text': '것', 'value': 10}, {'text': '사이즈', '..."
...,...,...
23550,978152,"[{'text': '가격', 'value': 1}, {'text': '질도', 'v..."
23551,981292,"[{'text': '제일', 'value': 1}, {'text': '상품', 'v..."
23552,981585,"[{'text': '구매', 'value': 2}, {'text': '사이즈', '..."
23553,987007,[]


In [326]:
sub = pd.merge(left=sub, right=k, how="left", on='goodsNo')
sub

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun
0,1766979,L,176697978839,78839,"[{'text': '옷', 'value': 3}, {'text': '과', 'val..."
1,2391602,FREE,239160278840,78840,"[{'text': '색상', 'value': 1}, {'text': '상품', 'v..."
2,1962220,black/free,196222078841,78841,"[{'text': '재질', 'value': 1}, {'text': '보풀', 'v..."
3,2252652,레드/L,225265278842,78842,"[{'text': '구명조끼', 'value': 2}, {'text': '빨간색',..."
4,1549580,FREE,154958078843,78843,"[{'text': '아이보리', 'value': 5}, {'text': '색', '..."
...,...,...,...,...,...
52541,1229923,XL,1229923131380,131380,"[{'text': '조금', 'value': 1}, {'text': '편이', 'v..."
52542,2107087,FREE,2107087131381,131381,"[{'text': '색감', 'value': 3}, {'text': '짜임', 'v..."
52543,1653581,M,1653581131382,131382,"[{'text': '털', 'value': 2}, {'text': '나름', 'va..."
52544,1653581,L,1653581131383,131383,"[{'text': '털', 'value': 2}, {'text': '나름', 'va..."


In [327]:
d = df_outer.groupby(['goodsNo', 'goodsSize']).sum()[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]
d

month1  month2  month3  month4  month5  month6  \
goodsNo goodsSize                                                          
18798   L                      0       0       0       0       0       0   
50937   S                      0       0       0       2       0       0   
        XL                     0       0       0       0       0       0   
55782   38                     1       0       0       0       0       0   
        42                     1       0       0       0       0       0   
...                          ...     ...     ...     ...     ...     ...   
981585  레드/M                   0       0       0       1       0       0   
        레드/XL                  0       0       0       1       0       0   
987007  INCENSEBEIGE/090       0       0       1       0       0       0   
992189  RESLAXED[L]            0       0       0       0       0       0   
        SIGNATURE[XL]          0       0       0       0       0       0   

                          month7  month8  month9  month10  month11  month12  
goodsNo goodsSize                                                            
18798   L                      0       0       0        1        0        0  
50937   S                      0       0       1        0        0        1  
        XL                     0       0       0        0        1        0  
55782   38                     0       0       0        0        1        0  
        42                     0       0       0        0        2        0  
...                          ...     ...     ...      ...      ...      ...  
981585  레드/M                   0       0       0        0        0        0  
        레드/XL                  0       0       0        0        0        0  
987007  INCENSEBEIGE/090       0       0       0        0        0        0  
992189  RESLAXED[L]            0       0       1        0        0        0  
        SIGNATURE[XL]          0       1       0        0        0        0  

[52546 rows x 12 columns]

#### 키, 몸무게 평균 구하기

In [328]:
c = df_outer.groupby(['goodsNo', 'goodsSize'], as_index=False).mean()[['goodsNo', 'goodsSize', 'userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]
c

,goodsNo,goodsSize,userHeight,userWeight,userSexMen,userSexWomen
0,18798,L,176.000000,68.000000,1.0,0.0
1,50937,S,167.500000,54.500000,0.5,0.5
2,50937,XL,171.000000,68.000000,1.0,0.0
3,55782,38,171.500000,82.500000,1.0,0.0
4,55782,42,176.333333,80.666667,1.0,0.0
...,...,...,...,...,...,...
52541,981585,레드/M,176.000000,65.000000,1.0,0.0
52542,981585,레드/XL,182.000000,77.000000,1.0,0.0
52543,987007,INCENSEBEIGE/090,160.000000,60.000000,0.0,1.0
52544,992189,RESLAXED[L],168.000000,54.000000,1.0,0.0


In [329]:
sub.sort_values(['goodsNo', 'goodsSize'], inplace=True)
c.sort_values(['goodsNo', 'goodsSize'], inplace=True)
d.sort_values(['goodsNo', 'goodsSize'], inplace=True)
sub.index = range(len(d))

In [330]:
d.index = range(len(d))
d

,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,2,0,0,0,0,1,0,0,1
2,0,0,0,0,0,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
52541,0,0,0,1,0,0,0,0,0,0,0,0
52542,0,0,0,1,0,0,0,0,0,0,0,0
52543,0,0,1,0,0,0,0,0,0,0,0,0
52544,0,0,0,0,0,0,0,0,1,0,0,0


In [331]:
sub = pd.concat([sub, c[['userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]], axis=1)
sub = pd.concat([sub, d[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]], axis=1)
sub.index = range(len(sub))
sub

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,18798,L,187980097723,97723,"[{'text': '몸', 'value': 1}, {'text': '모양', 'va...",176.000000,68.000000,1.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0
1,50937,S,5093700101241,101241,"[{'text': '가격', 'value': 2}, {'text': '옷', 'va...",167.500000,54.500000,0.5,0.5,0,0,0,2,0,0,0,0,1,0,0,1
2,50937,XL,5093700101242,101242,"[{'text': '가격', 'value': 2}, {'text': '옷', 'va...",171.000000,68.000000,1.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0
3,55782,38,5578200118983,118983,"[{'text': '옷', 'value': 2}, {'text': '가격', 'va...",171.500000,82.500000,1.0,0.0,1,0,0,0,0,0,0,0,0,0,1,0
4,55782,42,5578200118982,118982,"[{'text': '옷', 'value': 2}, {'text': '가격', 'va...",176.333333,80.666667,1.0,0.0,1,0,0,0,0,0,0,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52541,2408513,WHITE/FREE,240851385961,85961,"[{'text': '링클', 'value': 1}, {'text': '느낌', 'v...",176.000000,65.000000,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0
52542,2408513,WHITE/FREE,2408513110869,110869,"[{'text': '링클', 'value': 1}, {'text': '느낌', 'v...",182.000000,77.000000,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0
52543,2408664,DCH0_DARK-CHARCOL/095,2408664110810,110810,"[{'text': '디자인', 'value': 1}, {'text': '색감', '...",160.000000,60.000000,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0
52544,2410723,M,241072382324,82324,"[{'text': '개', 'value': 1}, {'text': '생각', 'va...",168.000000,54.000000,1.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0


In [115]:
df_outer = pd.merge(left=df_outer, right=sub[['goodsNo', 'goodsSize','newGoodsNo']], how="left", on=['goodsNo', 'goodsSize'])
df_outer.head()

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,뉴비_8e8c2e52077b,2021,9,18,1766979,1,0,180,61,L,"[제가 어두운 옷보단 밝은 옷을 좋아하는데, 너무 밝지도 않고 딱 적당합니다., 안...",https://image.msscdn.net/data/estimate/1766979...,1,2,1,2,2,0.0,0.0,0.0,0.0,1.766980e+11
1,무신가신사,2022,3,15,2391602,0,1,165,46,FREE,[예뻐요 색상도 엄청 환해지고 상품이 봤을때 더 예뻐요],-,0,2,2,2,2,0.0,2.0,0.0,0.0,2.391603e+11
2,뉴비_3c17ca57,2021,7,11,1962220,0,1,163,54,black/free,[재질이 보풀이 조금 생길거 같지만 부들부들 너무 좋네요 ㅠ 다른 색상도 구매 하려구요],-,0,2,2,2,2,0.0,2.0,0.0,0.0,1.962221e+11
3,푸쳐핸섭,2022,2,20,2252652,1,0,175,78,레드/L,"[빨간색 좋아해서 삿는뎅, 맘에는 드는뎅, 구명조끼 같다는 동료의 말에 빵터짐, 필...",-,0,2,2,1,1,0.0,0.0,0.0,0.0,2.252653e+11
4,mmd99,2021,12,10,1549580,1,0,178,90,FREE,[이뻐요 따듯해요 네이비 색상도 꼭 사고싶네요 굿굳!],https://image.msscdn.net/data/estimate/1549580...,1,1,1,1,1,0.0,1.0,0.0,0.0,1.549581e+11


In [333]:
len(sub['goodsNo'].unique())

21868

In [116]:
df_outer.shape

(911573, 22)

In [335]:
sub.to_csv('size_outer.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

In [336]:
sub.to_json('size_outer.json', orient='records', force_ascii=False)

In [117]:
df_outer.to_json('review_outer.json', orient='records', force_ascii=False)

### 그 외

In [876]:
df_outer['size'].value_counts()

2    712722
1    184071
3     14742
0        38
Name: size, dtype: int64

In [877]:
df_outer.bright.value_counts()

2    742704
1     89355
3     79475
0        39
Name: bright, dtype: int64

In [878]:
df_outer.color.value_counts()

2    674505
1    225746
3     11285
0        37
Name: color, dtype: int64

In [879]:
df_outer.thickness.value_counts()

2    594578
0    140032
1    110631
3     66332
Name: thickness, dtype: int64

In [880]:
df_outer.weightness.value_counts()

0.0    669783
2.0    156346
1.0     63524
3.0     21920
Name: weightness, dtype: int64

In [881]:
df_outer.touch.value_counts()

0.0    824217
2.0     52427
1.0     29786
3.0      5143
Name: touch, dtype: int64

### helpNo, styleLikeNo는 모두 0이다. 제거 가능성 높아짐

In [882]:
df_outer.describe()

,dateYear,dateMonth,dateDay,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
count,911573.000000,911573.000000,911573.000000,911573.000000,911573.000000,911573.000000,911573.000000,911573.000000,911573.000000,911573.000000,911573.000000,911573.000000,911573.000000,911573.000000,911573.000000,911573.000000,911573.000000,911573.000000,9.115730e+05,911573.000000,911573.000000,911573.000000,911573.000000,911573.000000,911573.00000,911573.000000,911573.0,911573.0,8.729770e+05
mean,2021.137822,6.891582,15.354941,0.110030,0.129391,0.125687,0.047659,0.030472,0.023074,0.014335,0.022153,0.058200,0.120006,0.165083,0.153911,0.749746,0.250254,172.440701,3.245750e+02,0.280736,1.814162,1.989076,1.764654,1.644172,0.48485,0.164626,0.0,0.0,1.495185e+12
std,0.606000,4.194411,8.706850,0.312927,0.335632,0.331497,0.213045,0.171881,0.150140,0.118866,0.147181,0.234122,0.324968,0.371255,0.360863,0.433160,0.433160,86.685594,1.739635e+05,0.449359,0.428637,0.430418,0.452547,0.825821,0.85853,0.535165,0.0,0.0,2.012530e+12
min,2020.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,1.001430e+11
25%,2021.000000,3.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,168.000000,6.000000e+01,0.000000,2.000000,2.000000,2.000000,1.000000,0.00000,0.000000,0.0,0.0,1.778798e+11
50%,2021.000000,8.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,173.000000,6.800000e+01,0.000000,2.000000,2.000000,2.000000,2.000000,0.00000,0.000000,0.0,0.0,1.134355e+12
75%,2022.000000,11.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,178.000000,7.500000e+01,1.000000,2.000000,2.000000,2.000000,2.000000,1.00000,0.000000,0.0,0.0,1.803462e+12
max,2022.000000,12.000000,31.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,82523.000000,1.234568e+08,1.000000,3.000000,3.000000,3.000000,3.000000,3.00000,3.000000,0.0,0.0,9.979720e+12


In [974]:
df_outer.to_csv('review_outer.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

# TOP 리뷰 전처리

In [190]:
df_top1_1 = pd.read_csv('./musinsa_review_top1_1.csv')
df_top1_2 = pd.read_csv('./musinsa_review_top1_2.csv', encoding='cp949')
df_top1_3 = pd.read_csv('./musinsa_review_top1_3.csv', encoding='cp949')
df_top2_1 = pd.read_csv('./musinsa_review_top2_1.csv')
df_top2_2 = pd.read_csv('./musinsa_review_top2_2.csv')
df_top2_3 = pd.read_csv('./musinsa_review_top2_3.csv')
df_top3_1 = pd.read_csv('./musinsa_review_top3_1.csv')
df_top3_2 = pd.read_csv('./musinsa_review_top3_2.csv')
df_top3_3 = pd.read_csv('./musinsa_review_top3_3.csv')
df_top3_4 = pd.read_csv('./musinsa_review_top3_4.csv')
df_top3_5 = pd.read_csv('./musinsa_review_top3_5.csv')
df_top4_1 = pd.read_csv('./musinsa_review_top4_1.csv')
df_top4_2 = pd.read_csv('./musinsa_review_top4_2.csv')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12588\2961329832.py:3: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_top1_3 = pd.read_csv('./musinsa_review_top1_3.csv', encoding='cp949')


In [191]:
print(df_top1_1.shape, df_top1_2.shape, df_top1_3.shape, df_top2_1.shape, df_top2_2.shape, df_top2_3.shape, df_top3_1.shape, df_top3_2.shape, df_top3_3.shape, df_top3_4.shape, df_top3_5.shape, df_top4_1.shape, df_top4_2.shape)

(282510, 19) (137842, 19) (209770, 19) (141551, 19) (174419, 19) (48441, 19) (70205, 19) (56159, 19) (59067, 19) (59259, 19) (90934, 19) (457875, 19) (14749, 19)


In [192]:
282510+137842+209770+141551+174419+48441+70205+56159+59067+59259+90934+457875+14749

1802781

In [193]:
df_top = pd.concat([df_top1_1, df_top1_2, df_top1_3, df_top2_1, df_top2_2, df_top2_3, df_top3_1, df_top3_2, df_top3_3, df_top3_4, df_top3_5, df_top4_1, df_top4_2], axis=0)
df_top.shape

(1802781, 19)

In [194]:
df_top

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,sbom,2022.01.04,2062047,0,1,168cm,67kg,1(ONESIZE),['음....어깨가 광활해서 남자옷위주로입는데 이건 사이즈가 크고 기장도길고 해서 ...,NaN,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0
1,mishoww,2021.11.19,2062047,0,1,154cm,50kg,1(ONESIZE),['길이가 적당하니 레깅스에 입어도 이쁘네요 만족합니다.'],NaN,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0
2,저리치워,2021.08.27,2062047,0,1,171cm,54kg,1(ONESIZE),"['프린팅 귀엽고 핏도 예뻐요 근데 목부분이 약해서 금방 늘어날 것 같은 느낌,,,']",NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
3,하아안빈,2022.01.10,2085464,1,0,171cm,60kg,IVORY/M,['안얇고 따뜻해서 좋아요 안에 티 하나 이거 하나 패딩 하나 이케하면 딱'],https://image.msscdn.net/data/estimate/2085464...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4,하아안빈,2022.01.10,2085464,1,0,171cm,60kg,IVORY/M,['안얇고 따뜻하니 좋아요 사이즈 좀 크니까 실제 사이즈 보고 사세요'],NaN,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14744,뉴비_e8fb6ff,2022.03.04,2331506,1,0,177cm,74kg,100,['이상적인 검정 바지 찾았네요ㅎㅎ 적당히 루즈핏이라 이번 봄 잘입고 다닐 거 같습...,NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
14745,오이파리,2022.03.13,2331511,1,0,180cm,70kg,100,"['두껍지않아서 지금입기 딱 좋아요', <br/>, '정사이즈 넉넉하게 맞습니다']",NaN,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0
14746,불꽃비형,2022.03.07,2331511,1,0,171cm,70kg,095,['컬러감 좋고 사이즈 좋고 생각보다 얇아요 핏은 이뻐요......'],NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
14747,goooooood!,2022.02.27,2331511,1,0,171cm,65kg,095,"['정사이즈 ~세미오버핏 중간 사이즈입니다~~', <br/>, '색감 쨍하니 좋습니다']",NaN,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0


In [195]:
df_top.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1802781 entries, 0 to 14748
Data columns (total 19 columns):
 #   Column         Dtype  
---  ------         -----  
 0   userName       object 
 1   date           object 
 2   goodsNo        object 
 3   userSexMen     object 
 4   userSexWomen   object 
 5   userHeight     object 
 6   userWeight     object 
 7   goodsSize      object 
 8   reviewContent  object 
 9   reviewImg      object 
 10  reviewStyle    float64
 11  size           float64
 12  bright         float64
 13  color          float64
 14  thickness      float64
 15  weightness     float64
 16  touch          float64
 17  helpNo         float64
 18  styleLikeNo    float64
dtypes: float64(9), object(10)
memory usage: 275.1+ MB


In [196]:
df_top.shape

(1802781, 19)

### 탈퇴 인원 제거

In [197]:
df_top.dropna(subset="userName", axis=0, inplace=True)
df_top = df_top.reset_index(drop=True)

In [198]:
df_top.shape

(1800968, 19)

### Date를 년, 월, 일로 등분

In [199]:
df_top.dropna(subset="date", axis=0, inplace=True)

In [200]:
df_top.shape

(1800968, 19)

In [201]:
df_top.insert(2, "dateDay", 0, True)
df_top.insert(2, "dateMonth", 0, True)
df_top.insert(2, "dateYear", 0, True)

In [202]:
df_top.columns

Index(['userName', 'date', 'dateYear', 'dateMonth', 'dateDay', 'goodsNo',
       'userSexMen', 'userSexWomen', 'userHeight', 'userWeight', 'goodsSize',
       'reviewContent', 'reviewImg', 'reviewStyle', 'size', 'bright', 'color',
       'thickness', 'weightness', 'touch', 'helpNo', 'styleLikeNo'],
      dtype='object')

In [203]:
result = []
for i in range(len(df_top)):
    if len(df_top.date[i].split('.')) != 3:
        result.append(i)
result

[314795, 316631, 382092, 461969, 963837, 1138437]

In [204]:
df_top = df_top.drop(result)
df_top.reset_index(drop=True)

,userName,date,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,sbom,2022.01.04,0,0,0,2062047,0,1,168cm,67kg,1(ONESIZE),['음....어깨가 광활해서 남자옷위주로입는데 이건 사이즈가 크고 기장도길고 해서 ...,NaN,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0
1,mishoww,2021.11.19,0,0,0,2062047,0,1,154cm,50kg,1(ONESIZE),['길이가 적당하니 레깅스에 입어도 이쁘네요 만족합니다.'],NaN,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0
2,저리치워,2021.08.27,0,0,0,2062047,0,1,171cm,54kg,1(ONESIZE),"['프린팅 귀엽고 핏도 예뻐요 근데 목부분이 약해서 금방 늘어날 것 같은 느낌,,,']",NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
3,하아안빈,2022.01.10,0,0,0,2085464,1,0,171cm,60kg,IVORY/M,['안얇고 따뜻해서 좋아요 안에 티 하나 이거 하나 패딩 하나 이케하면 딱'],https://image.msscdn.net/data/estimate/2085464...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4,하아안빈,2022.01.10,0,0,0,2085464,1,0,171cm,60kg,IVORY/M,['안얇고 따뜻하니 좋아요 사이즈 좀 크니까 실제 사이즈 보고 사세요'],NaN,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800957,뉴비_e8fb6ff,2022.03.04,0,0,0,2331506,1,0,177cm,74kg,100,['이상적인 검정 바지 찾았네요ㅎㅎ 적당히 루즈핏이라 이번 봄 잘입고 다닐 거 같습...,NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
1800958,오이파리,2022.03.13,0,0,0,2331511,1,0,180cm,70kg,100,"['두껍지않아서 지금입기 딱 좋아요', <br/>, '정사이즈 넉넉하게 맞습니다']",NaN,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0
1800959,불꽃비형,2022.03.07,0,0,0,2331511,1,0,171cm,70kg,095,['컬러감 좋고 사이즈 좋고 생각보다 얇아요 핏은 이뻐요......'],NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
1800960,goooooood!,2022.02.27,0,0,0,2331511,1,0,171cm,65kg,095,"['정사이즈 ~세미오버핏 중간 사이즈입니다~~', <br/>, '색감 쨍하니 좋습니다']",NaN,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0


In [205]:
df_top['dateYear'] = df_top.apply(lambda x: x['date'].split('.')[0], axis=1)
df_top['dateMonth'] = df_top.apply(lambda x: x['date'].split('.')[1], axis=1)
df_top['dateDay'] = df_top.apply(lambda x: x['date'].split('.')[2], axis=1)

In [206]:
df_top.shape

(1800962, 22)

In [207]:
df_top = df_top.drop('date', axis=1)

In [208]:
df_top.dateYear = df_top.dateYear.astype('int')
df_top.dateMonth = df_top.dateMonth.astype('int')
df_top.dateDay = df_top.dateDay.astype('int')

### goodsNo

In [209]:
df_top.dropna(subset="goodsNo", axis=0, inplace=True)

In [210]:
df_top.shape

(1800962, 21)

### 유저 성별

In [211]:
df_top.userSexMen = df_top.userSexMen.astype('int')
df_top.userSexWomen = df_top.userSexWomen.astype('int')

In [212]:
df_top[df_top.userSexMen == 1].shape

(1317627, 21)

In [213]:
df_top[df_top.userSexWomen==1].shape

(483215, 21)

In [214]:
a = list((df_top[df_top.userSexMen==0][df_top.userSexWomen==0]).index)
len(a)

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12588\2693055301.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = list((df_top[df_top.userSexMen==0][df_top.userSexWomen==0]).index)


120

In [215]:
df_top = df_top.drop(a)
df_top.reset_index(drop=True)

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,sbom,2022,1,4,2062047,0,1,168cm,67kg,1(ONESIZE),['음....어깨가 광활해서 남자옷위주로입는데 이건 사이즈가 크고 기장도길고 해서 ...,NaN,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0
1,mishoww,2021,11,19,2062047,0,1,154cm,50kg,1(ONESIZE),['길이가 적당하니 레깅스에 입어도 이쁘네요 만족합니다.'],NaN,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0
2,저리치워,2021,8,27,2062047,0,1,171cm,54kg,1(ONESIZE),"['프린팅 귀엽고 핏도 예뻐요 근데 목부분이 약해서 금방 늘어날 것 같은 느낌,,,']",NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
3,하아안빈,2022,1,10,2085464,1,0,171cm,60kg,IVORY/M,['안얇고 따뜻해서 좋아요 안에 티 하나 이거 하나 패딩 하나 이케하면 딱'],https://image.msscdn.net/data/estimate/2085464...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4,하아안빈,2022,1,10,2085464,1,0,171cm,60kg,IVORY/M,['안얇고 따뜻하니 좋아요 사이즈 좀 크니까 실제 사이즈 보고 사세요'],NaN,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800837,뉴비_e8fb6ff,2022,3,4,2331506,1,0,177cm,74kg,100,['이상적인 검정 바지 찾았네요ㅎㅎ 적당히 루즈핏이라 이번 봄 잘입고 다닐 거 같습...,NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
1800838,오이파리,2022,3,13,2331511,1,0,180cm,70kg,100,"['두껍지않아서 지금입기 딱 좋아요', <br/>, '정사이즈 넉넉하게 맞습니다']",NaN,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0
1800839,불꽃비형,2022,3,7,2331511,1,0,171cm,70kg,095,['컬러감 좋고 사이즈 좋고 생각보다 얇아요 핏은 이뻐요......'],NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
1800840,goooooood!,2022,2,27,2331511,1,0,171cm,65kg,095,"['정사이즈 ~세미오버핏 중간 사이즈입니다~~', <br/>, '색감 쨍하니 좋습니다']",NaN,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0


In [216]:
df_top.shape

(1800842, 21)

### 키와 몸무게

In [217]:
df_top.dropna(subset=["userHeight", "userWeight"], axis=0, inplace=True)

In [218]:
df_top.shape

(1800842, 21)

In [219]:
df_top.userHeight = df_top.userHeight.apply(lambda x: x.replace("cm", "").replace(" ", ""))
df_top.userHeight = df_top.userHeight.astype('int64')

In [220]:
df_top.userWeight = df_top.userWeight.apply(lambda x: x.replace("kg", "").replace(" ", ""))
df_top.userWeight = df_top.userWeight.astype('int64')

In [221]:
a = list((df_top[df_top.userHeight==0][df_top.userWeight==0]).index)
len(a)

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12588\3749120873.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = list((df_top[df_top.userHeight==0][df_top.userWeight==0]).index)


97

In [222]:
df_top = df_top.drop(a)
df_top.reset_index(drop=True)

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,sbom,2022,1,4,2062047,0,1,168,67,1(ONESIZE),['음....어깨가 광활해서 남자옷위주로입는데 이건 사이즈가 크고 기장도길고 해서 ...,NaN,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0
1,mishoww,2021,11,19,2062047,0,1,154,50,1(ONESIZE),['길이가 적당하니 레깅스에 입어도 이쁘네요 만족합니다.'],NaN,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0
2,저리치워,2021,8,27,2062047,0,1,171,54,1(ONESIZE),"['프린팅 귀엽고 핏도 예뻐요 근데 목부분이 약해서 금방 늘어날 것 같은 느낌,,,']",NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
3,하아안빈,2022,1,10,2085464,1,0,171,60,IVORY/M,['안얇고 따뜻해서 좋아요 안에 티 하나 이거 하나 패딩 하나 이케하면 딱'],https://image.msscdn.net/data/estimate/2085464...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4,하아안빈,2022,1,10,2085464,1,0,171,60,IVORY/M,['안얇고 따뜻하니 좋아요 사이즈 좀 크니까 실제 사이즈 보고 사세요'],NaN,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800740,뉴비_e8fb6ff,2022,3,4,2331506,1,0,177,74,100,['이상적인 검정 바지 찾았네요ㅎㅎ 적당히 루즈핏이라 이번 봄 잘입고 다닐 거 같습...,NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
1800741,오이파리,2022,3,13,2331511,1,0,180,70,100,"['두껍지않아서 지금입기 딱 좋아요', <br/>, '정사이즈 넉넉하게 맞습니다']",NaN,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0
1800742,불꽃비형,2022,3,7,2331511,1,0,171,70,095,['컬러감 좋고 사이즈 좋고 생각보다 얇아요 핏은 이뻐요......'],NaN,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
1800743,goooooood!,2022,2,27,2331511,1,0,171,65,095,"['정사이즈 ~세미오버핏 중간 사이즈입니다~~', <br/>, '색감 쨍하니 좋습니다']",NaN,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0


In [223]:
df_top.shape

(1800745, 21)

### 이미지

In [224]:
df_top.reviewStyle.value_counts()

0.0    1386143
1.0     414602
Name: reviewStyle, dtype: int64

In [225]:
df_top.reviewImg = df_top.reviewImg.fillna("-")

In [226]:
df_top.reviewImg

0                                                          -
1                                                          -
2                                                          -
3          https://image.msscdn.net/data/estimate/2085464...
4                                                          -
                                 ...                        
1800963                                                    -
1800964                                                    -
1800965                                                    -
1800966                                                    -
1800967                                                    -
Name: reviewImg, Length: 1800745, dtype: object

### Contents

In [227]:
print(df_top.reviewContent[0])
print(df_top.reviewContent[1].split(","))

['음....어깨가 광활해서 남자옷위주로입는데 이건 사이즈가 크고 기장도길고 해서 오버핏으로 잘 맞을줄알았으나 정핏으로맞아오']
["['길이가 적당하니 레깅스에 입어도 이쁘네요 만족합니다.']"]


In [228]:
# span 태그가 들어가있는 리뷰내용 전처리

cnt = 0
result = []
for i in df_top.reviewContent:
    if '<span>' in i:
        cnt += 1
        result.append(i)
print(cnt)

34


In [229]:
for i in result:
    idx = df_top[df_top.reviewContent==i].index[0]
    df_top.loc[idx, 'reviewContent'] = i.split('</div')[0][:]

In [230]:
df_top.reviewContent = df_top.reviewContent.apply(lambda x: x.replace("[", "").replace("]", "").replace(" '", "").replace("'", "").replace(", <br/>", "").split(","))

In [231]:
print(df_top.reviewContent[0])
print(df_top.reviewContent[1])

['음....어깨가 광활해서 남자옷위주로입는데 이건 사이즈가 크고 기장도길고 해서 오버핏으로 잘 맞을줄알았으나 정핏으로맞아오']
['길이가 적당하니 레깅스에 입어도 이쁘네요 만족합니다.']


### 사이즈

In [232]:
sub = df_top[["goodsNo", "goodsSize"]]
sub = sub.drop_duplicates(subset = ["goodsNo", "goodsSize"]).reset_index(drop=True)
sub.dropna(subset="goodsNo", inplace=True)
sub.insert(2, 'newGoodsNo', sub.goodsNo, True)
sub.insert(3, 'index', 0, True)
sub['index'] = range(131385, 131385+len(sub))
sub.index = range(131385, 131385+len(sub))
# sub.newGoodsNo = sub.newGoodsNo.apply(lambda x: round(x))

In [233]:
sub.goodsNo = sub.goodsNo.astype('str')
sub['index'] = sub['index'].astype('str')
sub.newGoodsNo = sub.newGoodsNo.astype('str')

In [234]:
len(sub)

150230

In [235]:
for i in range(131385, len(sub)+131385):
    if len(sub.newGoodsNo[i]) < 7:
        while len(sub.newGoodsNo[i]) < 7:
            sub.newGoodsNo[i] += '0'
sub

,goodsNo,goodsSize,newGoodsNo,index
131385,2062047,1(ONESIZE),2062047,131385
131386,2085464,IVORY/M,2085464,131386
131387,2085464,IVORY/S,2085464,131387
131388,1984454,L,1984454,131388
131389,2287021,L,2287021,131389
...,...,...,...,...
281610,2331508,105,2331508,281610
281611,2331508,100,2331508,281611
281612,2331506,100,2331506,281612
281613,2331511,100,2331511,281613


In [236]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150230 entries, 131385 to 281614
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   goodsNo     150230 non-null  object
 1   goodsSize   150230 non-null  object
 2   newGoodsNo  150230 non-null  object
 3   index       150230 non-null  object
dtypes: object(4)
memory usage: 4.6+ MB


In [237]:
sub['newGoodsNo'] = sub.apply(lambda x: x['newGoodsNo']+x['index'], axis=1)
sub

,goodsNo,goodsSize,newGoodsNo,index
131385,2062047,1(ONESIZE),2062047131385,131385
131386,2085464,IVORY/M,2085464131386,131386
131387,2085464,IVORY/S,2085464131387,131387
131388,1984454,L,1984454131388,131388
131389,2287021,L,2287021131389,131389
...,...,...,...,...
281610,2331508,105,2331508281610,281610
281611,2331508,100,2331508281611,281611
281612,2331506,100,2331506281612,281612
281613,2331511,100,2331511281613,281613


In [238]:
sub.goodsNo = sub.goodsNo.astype('int')
sub.newGoodsNo = sub.newGoodsNo.astype('int64')

#### month 당 사는 옷 수

In [239]:
df_top.insert(5, 'month12', 0, True)
df_top.insert(5, 'month11', 0, True)
df_top.insert(5, 'month10', 0, True)
df_top.insert(5, 'month9', 0, True)
df_top.insert(5, 'month8', 0, True)
df_top.insert(5, 'month7', 0, True)
df_top.insert(5, 'month6', 0, True)
df_top.insert(5, 'month5', 0, True)
df_top.insert(5, 'month4', 0, True)
df_top.insert(5, 'month3', 0, True)
df_top.insert(5, 'month2', 0, True)
df_top.insert(5, 'month1', 0, True)

In [240]:
df_top.month1 = df_top.apply(lambda x: 1 if x.dateMonth==1 else x.month1, axis=1)
df_top.month2 = df_top.apply(lambda x: 1 if x.dateMonth==2 else x.month2, axis=1)
df_top.month3 = df_top.apply(lambda x: 1 if x.dateMonth==3 else x.month3, axis=1)
df_top.month4 = df_top.apply(lambda x: 1 if x.dateMonth==4 else x.month4, axis=1)
df_top.month5 = df_top.apply(lambda x: 1 if x.dateMonth==5 else x.month5, axis=1)
df_top.month6 = df_top.apply(lambda x: 1 if x.dateMonth==6 else x.month6, axis=1)
df_top.month7 = df_top.apply(lambda x: 1 if x.dateMonth==7 else x.month7, axis=1)
df_top.month8 = df_top.apply(lambda x: 1 if x.dateMonth==8 else x.month8, axis=1)
df_top.month9 = df_top.apply(lambda x: 1 if x.dateMonth==9 else x.month9, axis=1)
df_top.month10 = df_top.apply(lambda x: 1 if x.dateMonth==10 else x.month10, axis=1)
df_top.month11 = df_top.apply(lambda x: 1 if x.dateMonth==11 else x.month11, axis=1)
df_top.month12 = df_top.apply(lambda x: 1 if x.dateMonth==12 else x.month12, axis=1)

In [241]:
from copy import deepcopy
a = deepcopy(df_top)

In [242]:
k = a.groupby(['goodsNo'])['reviewContent'].apply(list).reset_index(name="reviewContents")
k

,goodsNo,reviewContents
0,10390,"[[문구가 너무 맘에 들어요, 시험기간에 입고다니기 딱 좋아요]]"
1,10396,[[스우파에서 엠마가 입었던 티랑 비슷한 느낌이어서 구매했어요 ㅎㅎ 팔쪽에 하얀색 ...
2,18716,"[[집에서 잠옷으로 입으려고 샀어요 얇고 잠옷으로 딱 좋네요], [집에서 잠옷으로 ..."
3,18724,[[외않사 너무 예쁨 제일 맘에듬 가성비최고ㅋㅋ 주변반응도 좋음 심플한데 특이해보인...
4,23466,[[너무 귀엽고 잘샀다고 생각합니다 흔치않은 디자인인데 깔끔해요]]
...,...,...
58262,979056,"[[핏도 이쁘고 착용감도 매우 편하고 데일리로 입기에 좋네요], [뭔가 생각 해 놓..."
58263,983276,"[[전체적으로 넉넉하고 루즈하게 입기 아주 좋습니다.], [예뻐요 같이 겹쳐입은거 ..."
58264,984295,"[[세미오버핏으로 잘맞습니다 추천드리겠습니다], [생각보다 엄청 얇아서 여름에도입기..."
58265,996189,"[[목부분이 엄청 튼튼해보여서 좋아요, 팔부분만 좀 작게나왔으면 더 좋을듯], [실..."


In [243]:
k.reviewContents = k.reviewContents.astype('str')
k.reviewContents = k.apply(lambda x: x.reviewContents.replace("[", "").replace("]", ""), axis=1)

In [244]:
k.insert(2, 'reviewNoun1', '', True)
k.reviewNoun1 = k.apply(lambda x: Counter(okt.nouns(x.reviewContents)).most_common(100), axis=1)
k

,goodsNo,reviewContents,reviewNoun1
0,10390,"'문구가 너무 맘에 들어요', '시험기간에 입고다니기 딱 좋아요'","[(문구, 1), (맘, 1), (시험, 1), (기간, 1)]"
1,10396,'스우파에서 엠마가 입었던 티랑 비슷한 느낌이어서 구매했어요 ㅎㅎ 팔쪽에 하얀색 프...,"[(구매, 2), (프린팅, 2), (스우파, 1), (엠마, 1), (티, 1),..."
2,18716,"'집에서 잠옷으로 입으려고 샀어요 얇고 잠옷으로 딱 좋네요', '집에서 잠옷으로 입...","[(잠옷, 4), (집, 2)]"
3,18724,'외않사 너무 예쁨 제일 맘에듬 가성비최고ㅋㅋ 주변반응도 좋음 심플한데 특이해보인대...,"[(외, 1), (제일, 1), (맘에듬, 1), (가성, 1), (비, 1), (..."
4,23466,'너무 귀엽고 잘샀다고 생각합니다 흔치않은 디자인인데 깔끔해요',"[(생각, 1), (치, 1), (디자인, 1)]"
...,...,...,...
58262,979056,"'핏도 이쁘고 착용감도 매우 편하고 데일리로 입기에 좋네요', '뭔가 생각 해 놓았...","[(뭔가, 2), (핏, 1), (착용, 1), (매우, 1), (데, 1), (일..."
58263,983276,"'전체적으로 넉넉하고 루즈하게 입기 아주 좋습니다.', '예뻐요 같이 겹쳐입은거 같...","[(전체, 1), (루즈, 1), (아주, 1), (겹, 1), (디테, 1), (..."
58264,984295,"'세미오버핏으로 잘맞습니다 추천드리겠습니다', '생각보다 엄청 얇아서 여름에도입기 ...","[(가격, 2), (세미, 1), (버핏, 1), (추천, 1), (생각, 1), ..."
58265,996189,"'목부분이 엄청 튼튼해보여서 좋아요', '팔부분만 좀 작게나왔으면 더 좋을듯', '...","[(사이즈, 52), (핏, 46), (가성, 46), (비, 39), (가격, 3..."


In [245]:
k.insert(3, 'reviewNoun', '', True)

In [246]:
for i in k.itertuples():
    kkk = []
    for x, y in k.reviewNoun1[i.Index]:
        kkkkk = {'text': x, 'value': y}
        kkk.append(kkkkk)
    k.at[i.Index, 'reviewNoun'] = kkk
k

,goodsNo,reviewContents,reviewNoun1,reviewNoun
0,10390,"'문구가 너무 맘에 들어요', '시험기간에 입고다니기 딱 좋아요'","[(문구, 1), (맘, 1), (시험, 1), (기간, 1)]","[{'text': '문구', 'value': 1}, {'text': '맘', 'va..."
1,10396,'스우파에서 엠마가 입었던 티랑 비슷한 느낌이어서 구매했어요 ㅎㅎ 팔쪽에 하얀색 프...,"[(구매, 2), (프린팅, 2), (스우파, 1), (엠마, 1), (티, 1),...","[{'text': '구매', 'value': 2}, {'text': '프린팅', '..."
2,18716,"'집에서 잠옷으로 입으려고 샀어요 얇고 잠옷으로 딱 좋네요', '집에서 잠옷으로 입...","[(잠옷, 4), (집, 2)]","[{'text': '잠옷', 'value': 4}, {'text': '집', 'va..."
3,18724,'외않사 너무 예쁨 제일 맘에듬 가성비최고ㅋㅋ 주변반응도 좋음 심플한데 특이해보인대...,"[(외, 1), (제일, 1), (맘에듬, 1), (가성, 1), (비, 1), (...","[{'text': '외', 'value': 1}, {'text': '제일', 'va..."
4,23466,'너무 귀엽고 잘샀다고 생각합니다 흔치않은 디자인인데 깔끔해요',"[(생각, 1), (치, 1), (디자인, 1)]","[{'text': '생각', 'value': 1}, {'text': '치', 'va..."
...,...,...,...,...
58262,979056,"'핏도 이쁘고 착용감도 매우 편하고 데일리로 입기에 좋네요', '뭔가 생각 해 놓았...","[(뭔가, 2), (핏, 1), (착용, 1), (매우, 1), (데, 1), (일...","[{'text': '뭔가', 'value': 2}, {'text': '핏', 'va..."
58263,983276,"'전체적으로 넉넉하고 루즈하게 입기 아주 좋습니다.', '예뻐요 같이 겹쳐입은거 같...","[(전체, 1), (루즈, 1), (아주, 1), (겹, 1), (디테, 1), (...","[{'text': '전체', 'value': 1}, {'text': '루즈', 'v..."
58264,984295,"'세미오버핏으로 잘맞습니다 추천드리겠습니다', '생각보다 엄청 얇아서 여름에도입기 ...","[(가격, 2), (세미, 1), (버핏, 1), (추천, 1), (생각, 1), ...","[{'text': '가격', 'value': 2}, {'text': '세미', 'v..."
58265,996189,"'목부분이 엄청 튼튼해보여서 좋아요', '팔부분만 좀 작게나왔으면 더 좋을듯', '...","[(사이즈, 52), (핏, 46), (가성, 46), (비, 39), (가격, 3...","[{'text': '사이즈', 'value': 52}, {'text': '핏', '..."


In [247]:
k = k.drop(['reviewContents', 'reviewNoun1'], axis=1)
k

,goodsNo,reviewNoun
0,10390,"[{'text': '문구', 'value': 1}, {'text': '맘', 'va..."
1,10396,"[{'text': '구매', 'value': 2}, {'text': '프린팅', '..."
2,18716,"[{'text': '잠옷', 'value': 4}, {'text': '집', 'va..."
3,18724,"[{'text': '외', 'value': 1}, {'text': '제일', 'va..."
4,23466,"[{'text': '생각', 'value': 1}, {'text': '치', 'va..."
...,...,...
58262,979056,"[{'text': '뭔가', 'value': 2}, {'text': '핏', 'va..."
58263,983276,"[{'text': '전체', 'value': 1}, {'text': '루즈', 'v..."
58264,984295,"[{'text': '가격', 'value': 2}, {'text': '세미', 'v..."
58265,996189,"[{'text': '사이즈', 'value': 52}, {'text': '핏', '..."


In [248]:
sub = pd.merge(left=sub, right=k, how="left", on='goodsNo')
sub

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun
0,2062047,1(ONESIZE),2062047131385,131385,"[{'text': '음', 'value': 1}, {'text': '어깨', 'va..."
1,2085464,IVORY/M,2085464131386,131386,"[{'text': '하나', 'value': 3}, {'text': '티', 'va..."
2,2085464,IVORY/S,2085464131387,131387,"[{'text': '하나', 'value': 3}, {'text': '티', 'va..."
3,1984454,L,1984454131388,131388,"[{'text': '프린팅', 'value': 1}, {'text': '사이즈', ..."
4,2287021,L,2287021131389,131389,"[{'text': '가격', 'value': 3}, {'text': '대비', 'v..."
...,...,...,...,...,...
150225,2331508,105,2331508281610,281610,"[{'text': '정사', 'value': 4}, {'text': '이즈', 'v..."
150226,2331508,100,2331508281611,281611,"[{'text': '정사', 'value': 4}, {'text': '이즈', 'v..."
150227,2331506,100,2331506281612,281612,"[{'text': '때', 'value': 2}, {'text': '거', 'val..."
150228,2331511,100,2331511281613,281613,"[{'text': '정사', 'value': 2}, {'text': '이즈', 'v..."


In [249]:
d = df_top.groupby(['goodsNo', 'goodsSize']).sum()[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]
d

month1  month2  month3  month4  month5  month6  month7  \
goodsNo goodsSize                                                           
10390   M               0       0       0       0       0       0       0   
10396   M               0       0       0       0       0       0       0   
18716   XXL             0       0       2       0       0       0       0   
18724   S               0       0       0       0       1       0       0   
23466   S               0       0       0       0       0       0       0   
...                   ...     ...     ...     ...     ...     ...     ...   
996517  L               0       1       0       1       2       4       1   
        M               0       0       1       3       2       2       4   
        S               0       0       0       0       0       1       0   
        XL              0       2       1       2       0       2       2   
        XXL             2       0       0       0       1       1       2   

                   month8  month9  month10  month11  month12  
goodsNo goodsSize                                             
10390   M               0       0        1        0        0  
10396   M               0       1        1        0        0  
18716   XXL             0       0        0        0        0  
18724   S               0       0        0        0        0  
23466   S               0       1        0        0        0  
...                   ...     ...      ...      ...      ...  
996517  L               1       0        4        0        2  
        M               0       1        4        2        0  
        S               0       0        3        1        0  
        XL              1       3        2        1        0  
        XXL             1       3        1        2        2  

[150230 rows x 12 columns]

#### 키, 몸무게 평균 구하기

In [250]:
c = df_top.groupby(['goodsNo', 'goodsSize'], as_index=False).mean()[['goodsNo', 'goodsSize', 'userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]
c

,goodsNo,goodsSize,userHeight,userWeight,userSexMen,userSexWomen
0,10390,M,160.000000,65.000000,0.0000,1.0000
1,10396,M,164.000000,55.000000,0.5000,0.5000
2,18716,XXL,175.000000,88.000000,1.0000,0.0000
3,18724,S,162.000000,48.000000,0.0000,1.0000
4,23466,S,160.000000,50.000000,1.0000,0.0000
...,...,...,...,...,...,...
150225,996517,L,176.375000,70.437500,0.9375,0.0625
150226,996517,M,171.315789,65.105263,1.0000,0.0000
150227,996517,S,168.400000,60.400000,0.8000,0.2000
150228,996517,XL,179.625000,77.625000,1.0000,0.0000


In [251]:
sub.sort_values(['goodsNo', 'goodsSize'], inplace=True)
c.sort_values(['goodsNo', 'goodsSize'], inplace=True)
d.sort_values(['goodsNo', 'goodsSize'], inplace=True)
sub.index = range(len(d))

In [252]:
d.index = range(len(d))
d

,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,1,1,0,0
2,0,0,2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
150225,0,1,0,1,2,4,1,1,0,4,0,2
150226,0,0,1,3,2,2,4,0,1,4,2,0
150227,0,0,0,0,0,1,0,0,0,3,1,0
150228,0,2,1,2,0,2,2,1,3,2,1,0


In [253]:
sub = pd.concat([sub, c[['userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]], axis=1)
sub = pd.concat([sub, d[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]], axis=1)
sub.index = range(len(sub))
sub

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,10390,M,1039000230446,230446,"[{'text': '문구', 'value': 1}, {'text': '맘', 'va...",160.000000,65.000000,0.0000,1.0000,0,0,0,0,0,0,0,0,0,1,0,0
1,10396,M,1039600198495,198495,"[{'text': '구매', 'value': 2}, {'text': '프린팅', '...",164.000000,55.000000,0.5000,0.5000,0,0,0,0,0,0,0,0,1,1,0,0
2,18716,XXL,1871600198467,198467,"[{'text': '잠옷', 'value': 4}, {'text': '집', 'va...",175.000000,88.000000,1.0000,0.0000,0,0,2,0,0,0,0,0,0,0,0,0
3,18724,S,1872400194701,194701,"[{'text': '외', 'value': 1}, {'text': '제일', 'va...",162.000000,48.000000,0.0000,1.0000,0,0,0,0,1,0,0,0,0,0,0,0
4,23466,S,2346600231696,231696,"[{'text': '생각', 'value': 1}, {'text': '치', 'va...",160.000000,50.000000,1.0000,0.0000,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150225,2408672,M,2408672154785,154785,"[{'text': '핏', 'value': 2}, {'text': '디테일', 'v...",176.375000,70.437500,0.9375,0.0625,0,1,0,1,2,4,1,1,0,4,0,2
150226,2408675,M,2408675213992,213992,"[{'text': '느낌', 'value': 6}, {'text': '스타일', '...",171.315789,65.105263,1.0000,0.0000,0,0,1,3,2,2,4,0,1,4,2,0
150227,2409261,WOMEN-ONE,2409261194715,194715,"[{'text': '유니크', 'value': 1}, {'text': '바로', '...",168.400000,60.400000,0.8000,0.2000,0,0,0,0,0,1,0,0,0,3,1,0
150228,2410927,beige/L,2410927248936,248936,"[{'text': '디자인', 'value': 2}, {'text': '앞', 'v...",179.625000,77.625000,1.0000,0.0000,0,2,1,2,0,2,2,1,3,2,1,0


In [254]:
df_top = pd.merge(left=df_top, right=sub[['goodsNo', 'goodsSize','newGoodsNo']], how="left", on=['goodsNo', 'goodsSize'])
df_top.head()

,userName,dateYear,dateMonth,dateDay,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,sbom,2022,1,4,2062047,1,0,0,0,0,0,0,0,0,0,0,0,0,1,168,67,1(ONESIZE),[음....어깨가 광활해서 남자옷위주로입는데 이건 사이즈가 크고 기장도길고 해서 오...,-,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,2.062047e+12
1,mishoww,2021,11,19,2062047,0,0,0,0,0,0,0,0,0,0,1,0,0,1,154,50,1(ONESIZE),[길이가 적당하니 레깅스에 입어도 이쁘네요 만족합니다.],-,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,2.062047e+12
2,저리치워,2021,8,27,2062047,0,0,0,0,0,0,0,1,0,0,0,0,0,1,171,54,1(ONESIZE),"[프린팅 귀엽고 핏도 예뻐요 근데 목부분이 약해서 금방 늘어날 것 같은 느낌, , , ]",-,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.062047e+12
3,하아안빈,2022,1,10,2085464,1,0,0,0,0,0,0,0,0,0,0,0,1,0,171,60,IVORY/M,[안얇고 따뜻해서 좋아요 안에 티 하나 이거 하나 패딩 하나 이케하면 딱],https://image.msscdn.net/data/estimate/2085464...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2.085464e+12
4,하아안빈,2022,1,10,2085464,1,0,0,0,0,0,0,0,0,0,0,0,1,0,171,60,IVORY/M,[안얇고 따뜻하니 좋아요 사이즈 좀 크니까 실제 사이즈 보고 사세요],-,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2.085464e+12


In [255]:
len(sub['goodsNo'].unique())

58264

In [256]:
sub.shape

(150230, 21)

In [257]:
sub.to_csv('size_top.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

In [258]:
sub.to_json('size_top.json', orient='records', force_ascii=False)

### 그 외

In [259]:
df_top['size'].value_counts()

2.0    1390110
1.0     385968
3.0      24679
0.0         85
Name: size, dtype: int64

In [260]:
df_top.bright.value_counts()

2.0    1467134
1.0     241599
3.0      92022
0.0         87
Name: bright, dtype: int64

In [261]:
df_top.color.value_counts()

2.0    1289818
1.0     489291
3.0      21646
0.0         87
Name: color, dtype: int64

In [262]:
df_top.thickness.value_counts()

2.0    1425208
1.0     223137
3.0     152375
0.0        122
Name: thickness, dtype: int64

In [263]:
df_top.weightness.value_counts()

0.0    1800841
1.0          1
Name: weightness, dtype: int64

In [264]:
df_top.touch.value_counts()

0.0    1586749
2.0     126501
1.0      74252
3.0      13340
Name: touch, dtype: int64

### helpNo, styleLikeNo는 모두 0이다. 제거 가능성 높아짐

In [265]:
df_top.describe()

,dateYear,dateMonth,dateDay,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
count,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1.800842e+06,1800842.0,1800842.0,1.768123e+06
mean,2.021074e+03,7.074794e+00,1.534169e+01,8.184894e-02,9.438196e-02,8.496581e-02,4.045719e-02,4.791037e-02,6.638839e-02,6.892831e-02,7.230729e-02,8.773063e-02,1.214565e-01,1.190199e-01,1.146047e-01,7.316994e-01,2.683006e-01,1.724763e+02,1.586314e+03,2.302384e-01,1.799283e+00,1.916844e+00,1.740222e+00,1.960571e+00,5.552958e-07,2.039457e-01,0.0,0.0,2.380514e+12
std,5.519773e-01,3.683463e+00,8.789077e+00,2.741345e-01,2.923595e-01,2.788309e-01,1.970290e-01,2.135767e-01,2.489598e-01,2.533323e-01,2.589961e-01,2.829028e-01,3.266571e-01,3.238120e-01,3.185444e-01,4.430750e-01,4.430750e-01,2.153913e+02,1.611865e+06,4.209855e-01,4.335116e-01,4.225361e-01,4.652203e-01,4.552325e-01,7.451817e-04,5.893122e-01,0.0,0.0,2.021601e+12
min,2.020000e+03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,1.000039e+12
25%,2.021000e+03,3.000000e+00,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.670000e+02,5.800000e+01,0.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,1.473427e+12
50%,2.021000e+03,8.000000e+00,1.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.730000e+02,6.700000e+01,0.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,1.801584e+12
75%,2.021000e+03,1.000000e+01,2.300000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.780000e+02,7.500000e+01,0.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,2.086653e+12
max,2.022000e+03,1.200000e+01,3.100000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.801620e+05,2.147484e+09,1.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,1.000000e+00,3.000000e+00,0.0,0.0,9.999760e+12


In [266]:
df_top.to_csv('review_top.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')